# Steamboat Squad

Import and load data

In [1080]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1081]:
import json

with open("recipes_ingredients.json", "r") as json_file:
    recipes = json.load(json_file)
    
len(recipes)

4702

Overview of data structure. This is a list of dictionary, where each dictionary is a recipe with its name, ingredients and url

In [1082]:
recipes[0]

{'url': 'https://www.allrecipes.com/recipe/18318/pan-fried-asparagus/',
 'name': 'Pan-Fried Asparagus',
 'ingredients': ['¼ cup butter ',
  '2 tablespoons olive oil ',
  '1 teaspoon coarse salt ',
  '¼ teaspoon ground black pepper ',
  '3 cloves garlic, minced ',
  '1 pound fresh asparagus spears, trimmed ']}

Deleting url key

In [1083]:
for recipe in recipes:
    del recipe['url']
recipes[0]

{'name': 'Pan-Fried Asparagus',
 'ingredients': ['¼ cup butter ',
  '2 tablespoons olive oil ',
  '1 teaspoon coarse salt ',
  '¼ teaspoon ground black pepper ',
  '3 cloves garlic, minced ',
  '1 pound fresh asparagus spears, trimmed ']}

# Preprocessing Recipe Names
- Lower-casing (normalise words by using POS tagging)
- Change numbers to fix number (place holder)

NLTK has a help function that explains its POS tags.

In [1084]:
import nltk
from nltk import pos_tag, word_tokenize, RegexpParser, Tree
from nltk.tokenize import PunktSentenceTokenizer

nltk.download('tagsets')

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\tanke\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [1085]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

Using %%capture, save the NLTK help text as a string

In [1086]:
%%capture cap --no-stderr

nltk.help.upenn_tagset()

In [1087]:
cap.stdout

'$: dollar\n    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$\n\'\': closing quotation mark\n    \' \'\'\n(: opening parenthesis\n    ( [ {\n): closing parenthesis\n    ) ] }\n,: comma\n    ,\n--: dash\n    --\n.: sentence terminator\n    . ! ?\n:: colon or ellipsis\n    : ; ...\nCC: conjunction, coordinating\n    & \'n and both but either et for less minus neither nor or plus so\n    therefore times v. versus vs. whether yet\nCD: numeral, cardinal\n    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-\n    seven 1987 twenty \'79 zero two 78-degrees eighty-four IX \'60s .025\n    fifteen 271,124 dozen quintillion DM2,000 ...\nDT: determiner\n    all an another any both del each either every half la many much nary\n    neither no some such that the them these this those\nEX: existential there\n    there\nFW: foreign word\n    gemeinschaft hund ich jeux habeas Haementeria Herr K\'ang-si vous\n    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte\n    terram 

Using RE, get all the tag names

In [1088]:
import re

ALL_POS = re.findall(".*: +", cap.stdout)

for i, pos in enumerate(ALL_POS):
  ALL_POS[i] = pos.replace(': ', '')


ALL_POS

['$',
 "''",
 '(',
 ')',
 ',',
 '--',
 '.',
 ':',
 '    ',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'LS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'SYM',
 'TO',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WP$',
 'WRB',
 '``']

In [1089]:
ALL_POS.remove('    ')
ALL_POS

['$',
 "''",
 '(',
 ')',
 ',',
 '--',
 '.',
 ':',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'LS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'SYM',
 'TO',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WP$',
 'WRB',
 '``']

Create a function to pos tag a text

In [1090]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

def tag_pos(corpus):
    text=word_tokenize(corpus)
    return nltk.pos_tag(text)

tag_pos("This is a test sentence.")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tanke\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tanke\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('This', 'DT'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('test', 'NN'),
 ('sentence', 'NN'),
 ('.', '.')]

Create a function that POS tag and returns words with specific POS

In [1091]:
def get_words_with_pos(text, pos):
  tagged = tag_pos(text)
  return [t for t in tagged if t[1].startswith(pos)]

get_words_with_pos("This is a test sentence.", "NN")

[('test', 'NN'), ('sentence', 'NN')]

POS tag all recipe names

In [1092]:
tagged_recipe_names = []

for i, recipe in enumerate(recipes):
  try:
    tagged_recipe_names.append(tag_pos(recipes[i]['name']))
  except Exception as e:
    pass

len(tagged_recipe_names)

4701

## Data cleaning for names based on POS tagging

Looking at the first 10 tagged recipe names, there is a need for pre-processing, as NLTK's tagging is confused by the letter casing.

In [1093]:
tagged_recipe_names[:10]

[[('Pan-Fried', 'JJ'), ('Asparagus', 'NNP')],
 [('Pan', 'NNP'),
  ('de', 'FW'),
  ('Muertos', 'NNP'),
  ('(', '('),
  ('Mexican', 'NNP'),
  ('Bread', 'NNP'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('Dead', 'NNP'),
  (')', ')')],
 [('Creamy', 'NNP'), ('Au', 'NNP'), ('Gratin', 'NNP'), ('Potatoes', 'NNP')],
 [('Super-Delicious', 'JJ'), ('Zuppa', 'NNP'), ('Toscana', 'NNP')],
 [('Simple', 'JJ'), ('Teriyaki', 'NNP'), ('Sauce', 'NNP')],
 [('Spicy', 'JJ'),
  ('Korean', 'NNP'),
  ('Fried', 'NNP'),
  ('Chicken', 'NNP'),
  ('with', 'IN'),
  ('Gochujang', 'NNP'),
  ('Sauce', 'NNP')],
 [('Spaghetti', 'NNP'), ('Aglio', 'NNP'), ('e', 'NN'), ('Olio', 'NNP')],
 [('Easy', 'JJ'), ('Garam', 'NNP'), ('Masala', 'NNP')],
 [('Easy', 'NNP'), ('Chorizo', 'NNP'), ('Street', 'NNP'), ('Tacos', 'NNP')],
 [('Tres', 'NNS'),
  ('Leches', 'NNP'),
  ('(', '('),
  ('Milk', 'NNP'),
  ('Cake', 'NNP'),
  (')', ')')]]

Create a function that returns all tagged words with the same tag. NLTK's POS tagging assumes that capitalized noun means proper noun (name).

In [1094]:
def list_words_with_tag(tuple_list, pos):
  results = []
  for name in tuple_list:
    for tag in name:
      if tag[1] == pos:
        results.append(tag[0])
  return results

list_words_with_tag(tagged_recipe_names, "NNP")

['Asparagus',
 'Pan',
 'Muertos',
 'Mexican',
 'Bread',
 'Dead',
 'Creamy',
 'Au',
 'Gratin',
 'Potatoes',
 'Zuppa',
 'Toscana',
 'Teriyaki',
 'Sauce',
 'Korean',
 'Fried',
 'Chicken',
 'Gochujang',
 'Sauce',
 'Spaghetti',
 'Aglio',
 'Olio',
 'Garam',
 'Masala',
 'Easy',
 'Chorizo',
 'Street',
 'Tacos',
 'Leches',
 'Milk',
 'Cake',
 'Cabbage',
 'Rolls',
 'Gravy',
 'Shrimp',
 'Scampi',
 'Pasta',
 'Lemon',
 'Chicken',
 'Potato',
 'Bake',
 'Mexican',
 'Casserole',
 'Caldo',
 'Res',
 'Mexican',
 'Beef',
 'Soup',
 'Nogada',
 'Mexican',
 'Stuffed',
 'Poblano',
 'Peppers',
 'Walnut',
 'Sauce',
 'Apple',
 'Cake',
 'Flan',
 'Pork',
 'Chops',
 'Sauerkraut',
 'Spicy',
 'Thai',
 'Basil',
 'Chicken',
 'Pad',
 'Krapow',
 'Gai',
 'Spaghetti',
 'Cacio',
 'Pepe',
 'Chef',
 'John',
 'Chicken',
 'Kiev',
 'Chicken',
 'Onions',
 'Fajita',
 'Perfect',
 'Sushi',
 'Rice',
 'Baked',
 'Chicken',
 'German',
 'Potato',
 'Salad',
 'Miso',
 'Soup',
 'Mexican',
 'Rice',
 'II',
 'Haluski',
 'Labneh',
 'Lebanese',
 'Y

In [1095]:
all_name_tags = []

for POS in ALL_POS:
  new_dic = {POS: list_words_with_tag(tagged_recipe_names, POS)}
  all_name_tags.append(new_dic)

In [1096]:
def get_tag_number(tag_list):
  tag_numbers = []
  for tag in tag_list:
    for key, value in tag.items(): 
      new_dict = {key: len(value)}
    tag_numbers.append(new_dict)
  return tag_numbers

get_tag_number(all_name_tags)

[{'$': 1},
 {"''": 7},
 {'(': 529},
 {')': 529},
 {',': 63},
 {'--': 0},
 {'.': 10},
 {':': 98},
 {'CC': 555},
 {'CD': 74},
 {'DT': 104},
 {'EX': 0},
 {'FW': 47},
 {'IN': 482},
 {'JJ': 1822},
 {'JJR': 4},
 {'JJS': 27},
 {'LS': 0},
 {'MD': 2},
 {'NN': 571},
 {'NNP': 13139},
 {'NNPS': 46},
 {'NNS': 307},
 {'PDT': 0},
 {'POS': 348},
 {'PRP': 72},
 {'PRP$': 20},
 {'RB': 33},
 {'RBR': 0},
 {'RBS': 1},
 {'RP': 2},
 {'SYM': 0},
 {'TO': 20},
 {'UH': 0},
 {'VB': 24},
 {'VBD': 39},
 {'VBG': 50},
 {'VBN': 133},
 {'VBP': 10},
 {'VBZ': 22},
 {'WDT': 4},
 {'WP': 0},
 {'WP$': 0},
 {'WRB': 7},
 {'``': 6}]

Some names have numbers (CD). Some are obviously not numbers, like 'Figgy'

In [1097]:
def get_values_from_dict_list(dict_list, key):
  values = []
  for d in dict_list:
    if key in d:
      values.append(d[key])
  return values

cd_tokens = get_values_from_dict_list(all_name_tags, 'CD')[0]
cd_tokens

['5',
 '16',
 '2',
 '13',
 '300',
 'Figgy',
 '3',
 '9',
 'Two',
 '9',
 '22',
 '10',
 '15',
 'One',
 '18',
 'Ten',
 'Flounder',
 'Three',
 'Ziti',
 'One',
 '21',
 'Four',
 '9',
 '65',
 '17',
 '14',
 '10',
 "'n",
 '15',
 '8',
 'Minestrone',
 'Four',
 '35',
 'Fly',
 '15',
 '23',
 '8',
 '15',
 '21',
 "That's-a",
 'Tex-Mex',
 '14',
 '17',
 'Five',
 '10',
 '18',
 '5',
 "'Otai",
 '17',
 '3',
 '17',
 '75',
 '17',
 '20',
 'Take-Out',
 '16',
 '12',
 'Three',
 "'Three",
 '15',
 '20',
 '16',
 '12',
 '15',
 '22',
 '12',
 'Three',
 '21',
 '21',
 '25',
 '7',
 '10',
 '19',
 '20']

Create a function that searches for recipe name with specific string

In [1098]:
def find_value_with_char(dic_list, key, char):
  matches = []
  for recipe in dic_list:
    try:
      if char in recipe[key]:
        matches.append(recipe[key])
    except Exception as e:
      pass
  return matches

find_value_with_char(recipes, 'name', 'Figgy')

['Figgy Pudding']

'Three cup chicken' is indeed a name. On the other hand, numerics, such as 9 and 13 are not part of the actual names of dishes. So, numerics, instead of NLTK's CD, should be treated. This treatment should be done using regex.

In [1099]:
for cd in cd_tokens:
  print(find_value_with_char(recipes, 'name', cd))

['Our 5 Best Avgolemono Soup Recipes', '5-Ingredient Mexican Casserole', '15 Mexican-Inspired Ground Beef Casseroles That Deliver Big Flavor With Every Satisfying Bite', 'Chicken 65', 'Pan-Roasted 5-Spice Pork Loin', 'The 15 Most Iconic French Desserts', '35 Quick and Easy Chinese Dinners You Can Make at Home', '15 Essential North Indian Recipes', '15 Essential North Indian Recipes', '18 Easy Mexican Dishes With 5 Ingredients or Less', 'French 75 Cocktail', '15 Top-Rated Traditional German Christmas Cookies', '15 Traditional Italian Christmas Dinner Recipes', "25 Italian Cookies You'll Love"]
['16 German Recipes That Are Comfort Food Favorites', '16 Mexican-Inspired Casseroles for Family-Pleasing Dinners', '16 Essential Puerto Rican Recipes']
['2 Minute Cheese Quesadillas', "22 Recipes Using a Whole Baguette (That Aren't Sandwiches)", 'Our 21 Best Authentic Mexican Recipes', '23 Delicious Ways the World Cooks Pork Shoulder', '21 Easy Dinners That Start with Packaged Gnocchi', 'Our 20 B

Create a function that searches a regex pattern from a text

In [1100]:
def searchWordsPatt(text, patt):
    array = re.findall(patt, text)
    return array

NUMPATTERN = r'[0-9]+'
searchWordsPatt("I want 1 cup of tea", NUMPATTERN)

['1']

Create a function that substitutes regex patterns with a given value

In [1101]:
def searchReplacePatt(text, patt, new_val):
  return re.sub(patt, new_val, text)

NUMSPACEPATTERN = r'(\d+\s)'
searchReplacePatt("I want 1 cup of tea", NUMSPACEPATTERN, "")

'I want cup of tea'

searchReplacePatt, except it iterates recipe list

In [1102]:
def searchReplacePattList(dict_list, patt, new_val, key="name"):
    for i, recipe in enumerate(dict_list):
        try:
            dict_list[i][key] = searchReplacePatt(dict_list[i][key], patt, new_val)
        except Exception as e:
            pass

searchReplacePattList, but adds a substring at given index

In [1103]:
def searchReplaceAddPattList(dict_list, patt, new_val, substring, index=0, key="name"):
    for i, recipe in enumerate(dict_list):
        try:
            dict_list[i][key] = searchReplacePatt(dict_list[i][key], patt, new_val)
            added_string = list(dict_list[i][key]).insert(index, substring)
            dict_list[i][key]=''.join(added_string)
        except Exception as e:
            pass

Remove numerics from name

In [1104]:
import re

p_recipes = recipes

searchReplacePattList(p_recipes, NUMSPACEPATTERN, "")

def retag(text_list, key):
  new_list = []
  for i, recipe in enumerate(text_list):
    try:
      new_list.append(tag_pos(recipes[i][key]))
    except Exception as e:
      pass
  return new_list

tagged_recipe_names = retag(p_recipes, "name")

Get the new remaining CD

In [1105]:
new_cd_tokens = list_words_with_tag(tagged_recipe_names, "CD")
new_cd_tokens

['Figgy',
 'Two',
 'One',
 'Ten',
 'Flounder',
 'Three',
 'Ziti',
 'One',
 'Four',
 '65',
 "'n",
 'Minestrone',
 'Four',
 'Fly',
 "That's-a",
 'Tex-Mex',
 'Five',
 "'Otai",
 'Take-Out',
 'Three',
 "'Three",
 'Three']

The remaining numbers (CD) are part of actual recipe names

In [1106]:
for cd in new_cd_tokens:
  print(find_value_with_char(p_recipes, 'name', cd))

['Figgy Pudding']
['Two-Ingredient Naan', 'Pollo alla Birra for Two']
['A Number One Egg Bread', 'One-Egg Egg Drop Soup', 'One Pot Thai-Style Rice Noodles', 'One-Pot Vegan Potato-Lentil Curry', 'One-Bite Thai "Flavor Bomb" Salad Wraps (Miang Kham)', 'Easy One-Skillet Ground Beef Burrito', 'One-Pot Greek Lemon Chicken and Rice']
['Tender Italian Baked Chicken', 'Tuscan Pork Tenderloin', 'Asian Pork Tenderloin', 'Italian Pork Tenderloin', 'Sweet and Sour Pork Tenderloin', 'Chipotle Crusted Pork Tenderloin', 'Ten Minute Szechuan Chicken', 'Thai Quivering Tenderloins', 'Spicy Pork Tenderloin', 'Chinese Pork Tenderloin', 'Grecian Pork Tenderloin', 'Havana Slow Cooker Pork Tenderloin', 'Curry Pork Tenderloin', 'Tender Juicy Skirt Steak  (Churrasco)', 'Spicy and Tender Corned Beef', 'Pan Roasted Pork Tenderloin with a Blue Cheese and Olive Stuffing']
['Flounder Mediterranean']
['Pastel de Tres Leches (Three Milk Cake)', 'Three-Meat Italian Meatballs', 'Three Cheese Manicotti II', 'Taiwanese-S

In [1107]:
new_all_name_tags = []

for POS in ALL_POS:
  new_dic = {POS: list_words_with_tag(tagged_recipe_names, POS)}
  new_all_name_tags.append(new_dic)

Can and 'll are the modal verbs found

In [1108]:
md_tokens = list_words_with_tag(tagged_recipe_names, "MD")
md_tokens

['Can', "'ll"]

'can' is caused by words such as Canadian, which is processed in next section. But, 'you'll love' is not part of recipe name and more of an expression

In [1109]:
for md in md_tokens:
  print(find_value_with_char(p_recipes, 'name', md))

['Canadian Yellow Split Pea Soup with Ham', 'French Canadian Tourtiere', 'Pure Maple Candy', 'Cannoli', 'The Original Donair From the East Coast of Canada', 'Sauerkraut for Canning', 'Tourtiere (French Canadian Meat Pie)', 'Pumpkin Cannoli', 'Puerto Rican Canned Corned Beef Stew', 'Canadian Pork Loin Chops', 'Caneles de Bordeaux', 'Canadian Walleye (Pickerel)', "Thera's Canadian Fried Dough", 'Italian Baked Cannelloni', 'Canary Island Red Mojo Sauce', 'Mexican Tamarind Candy', 'Cantonese Chicken Chow Mein', 'Roti Canai/Paratha (Indian Pancake)', 'Polvorones de Canele (Cinnamon Cookies)', 'Miraculous Canadian Sugar Pie', 'Canadian Tea Biscuits', 'Peanut Butter Potato Candy', 'Irish Potato Candy', 'Filipino Pancit Bihon with Canton', 'Gorton (French-Canadian Pork Spread)', 'Quick and Easy Chinese Dinners You Can Make at Home', 'Chocolate Cantucci', 'Cantonese Style Lobster', 'Real Canadian Poutine', 'French Canadian Meatball Stew', 'Canadian Butter Tarts', 'Canadian Apple Pie', 'Cantones

Removing "You'll" and retagging new list

In [1110]:
searchReplacePattList(p_recipes, r"(You'll Love)", "")
tagged_recipe_names = retag(p_recipes, "name")

'll' removed

In [1111]:
new_md_tokens = list_words_with_tag(tagged_recipe_names, "MD")
new_md_tokens

['Can']

In [1112]:
for md in new_md_tokens:
  print(find_value_with_char(p_recipes, 'name', md))

['Canadian Yellow Split Pea Soup with Ham', 'French Canadian Tourtiere', 'Pure Maple Candy', 'Cannoli', 'The Original Donair From the East Coast of Canada', 'Sauerkraut for Canning', 'Tourtiere (French Canadian Meat Pie)', 'Pumpkin Cannoli', 'Puerto Rican Canned Corned Beef Stew', 'Canadian Pork Loin Chops', 'Caneles de Bordeaux', 'Canadian Walleye (Pickerel)', "Thera's Canadian Fried Dough", 'Italian Baked Cannelloni', 'Canary Island Red Mojo Sauce', 'Mexican Tamarind Candy', 'Cantonese Chicken Chow Mein', 'Roti Canai/Paratha (Indian Pancake)', 'Polvorones de Canele (Cinnamon Cookies)', 'Miraculous Canadian Sugar Pie', 'Canadian Tea Biscuits', 'Peanut Butter Potato Candy', 'Irish Potato Candy', 'Filipino Pancit Bihon with Canton', 'Gorton (French-Canadian Pork Spread)', 'Quick and Easy Chinese Dinners You Can Make at Home', 'Chocolate Cantucci', 'Cantonese Style Lobster', 'Real Canadian Poutine', 'French Canadian Meatball Stew', 'Canadian Butter Tarts', 'Canadian Apple Pie', 'Cantones

In [1113]:
searchReplacePattList(p_recipes, r"\/", " or ")
tagged_recipe_names = retag(p_recipes, "name")

In [1114]:
bracket_tokens = list(set(list_words_with_tag(tagged_recipe_names, "(")))
bracket_tokens

['(']

Examining brackers in names. Most of the words in brackets are translations

In [1115]:
bracketed_names = []
for bracket in bracket_tokens:
    names = find_value_with_char(p_recipes, 'name', bracket)
    print(names)
    bracketed_names = bracketed_names + names

bracketed_names = list(set(bracketed_names))

['Pan de Muertos (Mexican Bread of the Dead)', 'Tres Leches (Milk Cake)', 'Caldo de Res (Mexican Beef Soup)', 'Chiles en Nogada (Mexican Stuffed Poblano Peppers in Walnut Sauce)', 'Spicy Thai Basil Chicken (Pad Krapow Gai)', 'Labneh (Lebanese Yogurt)', 'Indian Chicken Curry (Murgh Kari)', 'Keema Aloo (Ground Beef and Potatoes)', 'Turkish Eggs (Cilbir)', 'South African Melktert (Milk Tart)', 'Ukrainian Apple Cake (Yabluchnyk)', 'Spanish Garlic Shrimp (Gambas al Ajillo)', 'Polish Noodles (Cottage Cheese and Noodles)', 'German Potato Dumplings (Kartoffelkloesse)', 'Apfelkuchen (Apple Cake)', 'Oyakodon (Japanese Chicken and Egg Rice Bowl)', 'Bibimbap (Korean Rice With Mixed Vegetables)', 'Eggplant Caponata (Sicilian Version)', 'Chana Masala (Savory Indian Chick Peas)', 'Ricotta Pie (Old Italian Recipe)', 'Easy Blini (Russian Pancake)', 'Easy Bulgogi (Korean BBQ Beef)', 'Carne en su Jugo (Meat in its Juices)', 'Ghormeh Sabzi (Persian Herb Stew)', 'Puerto Rican Tostones (Fried Plantains)', '

"(no red sauce here...golden)" needs to be removed

In [1116]:
# Redundant descriptions
searchReplacePattList(p_recipes,  r"(no red sauce here...golden)", "")
searchReplacePattList(p_recipes, r"(From a Swede!)", "")
searchReplacePattList(p_recipes, r"(from a Chinese person)", "")
searchReplacePattList(p_recipes, r"(Now Vegetarian!)", "")
searchReplacePattList(p_recipes, r"a.k.a. ", "")
searchReplacePattList(p_recipes, r"(That Aren't Sandwiches)", "")

# Remove copyright symbol
searchReplacePattList(p_recipes, r"&reg;", "")
# Asian Sesame Seared or Grilled Tuna (Gluten Free) => Gluten Free Asian Sesame Seared or Grilled Tuna
searchReplaceAddPattList(p_recipes, r"(Gluten Free)", "", "glutten-free")
tagged_recipe_names = retag(p_recipes, "name")

In [1117]:
bracketed_names = []
for bracket in bracket_tokens:
    names = find_value_with_char(p_recipes, 'name', bracket)
    print(names)
    bracketed_names = bracketed_names + names

bracketed_names = list(set(bracketed_names))

['Pan de Muertos (Mexican Bread of the Dead)', 'Tres Leches (Milk Cake)', 'Caldo de Res (Mexican Beef Soup)', 'Chiles en Nogada (Mexican Stuffed Poblano Peppers in Walnut Sauce)', 'Spicy Thai Basil Chicken (Pad Krapow Gai)', 'Labneh (Lebanese Yogurt)', 'Indian Chicken Curry (Murgh Kari)', 'Keema Aloo (Ground Beef and Potatoes)', 'Turkish Eggs (Cilbir)', 'South African Melktert (Milk Tart)', 'Ukrainian Apple Cake (Yabluchnyk)', 'Spanish Garlic Shrimp (Gambas al Ajillo)', 'Polish Noodles (Cottage Cheese and Noodles)', 'German Potato Dumplings (Kartoffelkloesse)', 'Apfelkuchen (Apple Cake)', 'Oyakodon (Japanese Chicken and Egg Rice Bowl)', 'Bibimbap (Korean Rice With Mixed Vegetables)', 'Eggplant Caponata (Sicilian Version)', 'Chana Masala (Savory Indian Chick Peas)', 'Ricotta Pie (Old Italian Recipe)', 'Easy Blini (Russian Pancake)', 'Easy Bulgogi (Korean BBQ Beef)', 'Carne en su Jugo (Meat in its Juices)', 'Ghormeh Sabzi (Persian Herb Stew)', 'Puerto Rican Tostones (Fried Plantains)', '

In [1118]:
fw_tokens = list(set(list_words_with_tag(tagged_recipe_names, "FW")))
fw_tokens

['et', 'Rassolnik', 'de']

In [1119]:
fw_names = []
for fw in fw_tokens:
    names = find_value_with_char(p_recipes, 'name', fw)
    print(names)
    fw_names = fw_names + names
fw_names = list(set(fw_names))

['Spaghetti Aglio e Olio', 'Easy Chorizo Street Tacos', 'Spaghetti Cacio e Pepe', 'Make-Ahead Vegetarian Moroccan Stew', "'Chinese Buffet' Green Beans", 'Sweet and Sour Chicken I', 'Bibimbap (Korean Rice With Mixed Vegetables)', 'Braised Corned Beef Brisket', 'Skillet Chicken Bulgogi', 'Easy Slow Cooker Chicken Tetrazzini', 'Fabulous Wet Burritos', 'Sauteed Sweet Plantains (Tajaditas Dulces de Platano)', 'Vegetarian Mexican Inspired Stuffed Peppers', 'Sheet Pan Chicken Fajitas', 'Sheet Pan Fried Rice', 'Vegetarian Chinese Fried Noodles', "Papa Drexler's Bavarian Pretzels", 'Quick Bruschetta Chicken Bake', 'Authentic Vietnamese Spring Rolls (Nem Ran Hay Cha Gio)', 'Kotlet Schabowy (Polish Breaded Pork Chop)', 'Spaghetti alla Carbonara: the Traditional Italian Recipe', 'Stir-Fry Chicken and Vegetables', 'Vegetarian Moussaka', 'French Baguettes', 'Shrimp Fettuccine Alfredo', 'Conchas (Mexican Sweet Bread)', 'Skillet Chicken Picante', 'Spaghetti Sauce', 'Roasted Pork Banh Mi (Vietnamese Sa

In [1120]:
fw_names

['Scandinavian Pickled Beets',
 'Sweet Tamale Pie',
 'Homemade Farfalle Pasta',
 'Spaghetti alla Nerano (Spaghetti with Fried Zucchini)',
 'Spicy Italian Pork Cutlets',
 'Homemade Muesli',
 'Indian Sweet Bread',
 'Spaetzle, Sauerkraut and Sausage Casserole',
 'Homemade Adobo All-Purpose Seasoning',
 'Homemade Italian Sausage',
 'Asado de Puerco (Mexican Pork Stew)',
 'Homemade Mince Pie with Crumbly Topping',
 'Brazilian Lemonade',
 'Kaese Spaetzle',
 'Chicken Enchiladas with Green Chile Sauce (Salsa Verde)',
 'Vegetarian Pad Thai',
 'Keftedes - Greek Meatballs',
 'Homemade Feta Cheese',
 'Vegetarian Mulligatawny Soup',
 'Beef with Vegetables',
 'Colorado Green Chili (Chile Verde)',
 'Garlic and Herb Marinade',
 'Lamb Feta Peppers',
 'Vegetable Curry',
 'Hot and Sweet Ginger Drink',
 'Creamy Chicken Marsala Fettuccine',
 'Spiced Sweet Roasted Red Pepper Hummus',
 'Shredded Beef Tacos with Lime',
 'An Easy-as-Pie Sweet and Sour Sauce ()',
 'Vietnamese Beef Pho',
 'Cheesy Vegetable Lasag

Names that both have foreign words and bracket

In [1121]:
bracket_and_fw = [name for name in bracketed_names if name in fw_names]
bracket_and_fw

['Spaghetti alla Nerano (Spaghetti with Fried Zucchini)',
 'Asado de Puerco (Mexican Pork Stew)',
 'Chicken Enchiladas with Green Chile Sauce (Salsa Verde)',
 'Colorado Green Chili (Chile Verde)',
 'An Easy-as-Pie Sweet and Sour Sauce ()',
 'Pebber Nodder (Danish Christmas Cookies)',
 'Quark (Homemade Cheese)',
 'Sauteed Sweet Plantains (Tajaditas Dulces de Platano)',
 'Agua de Jamaica (Hibiscus Water)',
 'Beef Barbacoa (Barbacoa de Res)',
 "Grandma's Povitisa (Po-vuh-teet-zuh) Povitica Croatian Nut Bread",
 'Postre de Limon (Mexican Lime Dessert)',
 'Philippine Longanisa de Eugenio (Sweet Sausage)',
 'Homemade Irish (Whiskey) Cream',
 'Tarte aux Moutarde (French Tomato and Mustard Pie)',
 'Birria de Chivo Estilo Jalisco (Mexican Braised Goat)',
 'Cazuela de Vaca (Beef and Pumpkin Stew)',
 'Brazilian Style Flan (Pudim de Leite Condensado)',
 'Kwek Kwek (Filipino Street Food)',
 'Pan de Muertos (Mexican Bread of the Dead)',
 'Vinagrete (Brazilian Tomato Slaw)',
 "Peposa Dell'Impruneta (

Split the names into two names, one outside and one inside

In [1122]:
BRACKET_REGEX = " \(.*\)"
def break_fw_bracket(name):
    name1 = re.findall(BRACKET_REGEX, name)[0]
    name1 = name1[name1.find("(")+1:name1.find(")")]
    name2 = re.sub(BRACKET_REGEX, "", name)
    return name1, name2

print(break_fw_bracket("Hearty Caldo de Res (Mexican Beef Soup)"))
print(break_fw_bracket("Ukha (Russian Fish Soup)"))

('Mexican Beef Soup', 'Hearty Caldo de Res')
('Russian Fish Soup', 'Ukha')


Apply the split function. Delete old recipe with bracket and foreign words. In both of the new recipes, duplicate old ingredients.

In [1123]:
for i, recipe in enumerate(p_recipes):
    try:
        if p_recipes[i]["name"] in bracket_and_fw:
            newname1, newname2 = break_fw_bracket(p_recipes[i]["name"])
            new_recipe1 = {'name': newname1, 'ingredients': p_recipes[i]["ingredients"]}
            new_recipe2 = {'name': newname2, 'ingredients': p_recipes[i]["ingredients"]}
            p_recipes.append(new_recipe1)
            p_recipes.append(new_recipe2)
            p_recipes.remove(p_recipes[i])
    except Exception as e:
        pass

tagged_recipe_names = retag(p_recipes, "name")

There are still remaining names with bracket, mostly due to the foreign words not being recognized.

In [1124]:
bracketed_names = []
for bracket in bracket_tokens:
    names = find_value_with_char(p_recipes, 'name', bracket)
    print(names)
    bracketed_names = bracketed_names + names

bracketed_names = list(set(bracketed_names))

['Tres Leches (Milk Cake)', 'Chiles en Nogada (Mexican Stuffed Poblano Peppers in Walnut Sauce)', 'Spicy Thai Basil Chicken (Pad Krapow Gai)', 'Labneh (Lebanese Yogurt)', 'Indian Chicken Curry (Murgh Kari)', 'Keema Aloo (Ground Beef and Potatoes)', 'Turkish Eggs (Cilbir)', 'South African Melktert (Milk Tart)', 'Ukrainian Apple Cake (Yabluchnyk)', 'Spanish Garlic Shrimp (Gambas al Ajillo)', 'Polish Noodles (Cottage Cheese and Noodles)', 'German Potato Dumplings (Kartoffelkloesse)', 'Apfelkuchen (Apple Cake)', 'Oyakodon (Japanese Chicken and Egg Rice Bowl)', 'Eggplant Caponata (Sicilian Version)', 'Chana Masala (Savory Indian Chick Peas)', 'Ricotta Pie (Old Italian Recipe)', 'Easy Blini (Russian Pancake)', 'Easy Bulgogi (Korean BBQ Beef)', 'Carne en su Jugo (Meat in its Juices)', 'Ghormeh Sabzi (Persian Herb Stew)', 'Puerto Rican Tostones (Fried Plantains)', 'Kalbi (Korean BBQ Short Ribs)', 'Macaron (French Macaroon)', 'Atsara (Papaya Relish)', 'Authentic Chinese Egg Rolls ()', 'Greek Le

In [1125]:
bracketed_names

['Japanese Egg Salad Sandwich (Tamago Sando)',
 'German Potato Dumplings (Kartoffelkloesse)',
 'Lagman (Uzbek Noodle Soup)',
 'Korean Spicy Marinated Pork (Dae Ji Bool Gogi)',
 "Dead Bones (L'Ossa Morte)",
 'Turkish Meatballs (Kofta)',
 'Instant Pot Galbi (Korean-Style Short Ribs)',
 'Afghan Beef Raviolis (Mantwo)',
 'Schwabischer Kartoffelsalat (German Potato Salad - Schwabisch Style)',
 'Tuscan Tomato Soup (Pappa al Pomodoro)',
 'Easy Bulgogi (Korean BBQ Beef)',
 'Blaukraut (German Red Cabbage)',
 'Sinigang na Isda sa Miso (Fish Stew with Miso)',
 'Dashi Stock (Konbudashi)',
 'Ginataang Alimasag (Crabs in Coconut Milk)',
 'Rahmschnitzel (German Schnitzel in Creamy Mushroom Sauce)',
 'One-Bite Thai "Flavor Bomb" Salad Wraps (Miang Kham)',
 'Chinese Barbeque Pork (Char Siu)',
 'Feijoada (Brazilian Black Bean Stew)',
 'Dak Bulgogi (Korean Barbeque Chicken)',
 'Turos Csusza (Pasta with Cottage Cheese)',
 "Lentil and Cactus Soup (Mom's Recipe)",
 'Marzipan Christmas Kringle (Juleskringle)

Most of the brackets are at the end of each name. For those that are in the middle, they are translations of one of the words in the name.

In [1126]:
b_name_end = []
b_name_mid = []
for b_name in bracketed_names:
    if b_name.endswith(')'):
        b_name_end.append(b_name)
    else:
        b_name_mid.append(b_name)
        
b_name_end

['Japanese Egg Salad Sandwich (Tamago Sando)',
 'German Potato Dumplings (Kartoffelkloesse)',
 'Lagman (Uzbek Noodle Soup)',
 'Korean Spicy Marinated Pork (Dae Ji Bool Gogi)',
 "Dead Bones (L'Ossa Morte)",
 'Turkish Meatballs (Kofta)',
 'Instant Pot Galbi (Korean-Style Short Ribs)',
 'Afghan Beef Raviolis (Mantwo)',
 'Schwabischer Kartoffelsalat (German Potato Salad - Schwabisch Style)',
 'Tuscan Tomato Soup (Pappa al Pomodoro)',
 'Easy Bulgogi (Korean BBQ Beef)',
 'Blaukraut (German Red Cabbage)',
 'Sinigang na Isda sa Miso (Fish Stew with Miso)',
 'Dashi Stock (Konbudashi)',
 'Ginataang Alimasag (Crabs in Coconut Milk)',
 'Rahmschnitzel (German Schnitzel in Creamy Mushroom Sauce)',
 'One-Bite Thai "Flavor Bomb" Salad Wraps (Miang Kham)',
 'Chinese Barbeque Pork (Char Siu)',
 'Feijoada (Brazilian Black Bean Stew)',
 'Dak Bulgogi (Korean Barbeque Chicken)',
 'Turos Csusza (Pasta with Cottage Cheese)',
 "Lentil and Cactus Soup (Mom's Recipe)",
 'Marzipan Christmas Kringle (Juleskringle)

In [1127]:
b_name_mid

['Lamb (Gosht) Biryani',
 'Karaage (Japanese Fried Chicken) with Honey Mayoster Sauce',
 'Lengua (Beef Tongue) Stew',
 'Jeera (Cumin) Rice',
 'Bee Sting Cake (Bienenstich) II',
 'Pollo (Chicken) Fricassee from Puerto Rico',
 'Fried Chicken Chunks (Chicharrones De Pollo) Dominican',
 'Hawaiian Bruddah Potato Mac (Macaroni) Salad',
 'Coconut (Haupia) and Chocolate Pie',
 'Seaweed (Nori) Soup',
 'Zito (Zhito or Koljivo) - Serbian Wheat Pudding',
 'Albondigas (Meatballs) en Chipotle',
 'Lazy Golumpki (Stuffed Cabbage) Soup',
 'Fusilli with Rapini (Broccoli Rabe), Garlic, and Tomato Wine Sauce',
 'Vareniki (Russian Pierogi) with Potatoes and Mushrooms',
 "World's Best () Lasagna",
 'Spicy Indian (Gujarati) Green Beans',
 'Korean Bean Curd (Miso) Soup',
 'Besan (Gram Flour) Halwa',
 'Kimchi Jun (Kimchi Pancake) and Dipping Sauce',
 'Fish Sinigang (Tilapia) - Filipino Sour Broth Dish',
 'Ulu (Breadfruit) Pancakes',
 'Classic Cuban Midnight (Medianoche) Sandwich']

On the other hand, without parenthesis anymore, names with foregin words tagged are now clean

In [1128]:
fw_names = []
for fw in fw_tokens:
    names = find_value_with_char(p_recipes, 'name', fw)
    print(names)
    fw_names = fw_names + names
fw_names = list(set(fw_names))

['Spaghetti Aglio e Olio', 'Easy Chorizo Street Tacos', 'Spaghetti Cacio e Pepe', 'Make-Ahead Vegetarian Moroccan Stew', "'Chinese Buffet' Green Beans", 'Sweet and Sour Chicken I', 'Braised Corned Beef Brisket', 'Skillet Chicken Bulgogi', 'Easy Slow Cooker Chicken Tetrazzini', 'Fabulous Wet Burritos', 'Vegetarian Mexican Inspired Stuffed Peppers', 'Sheet Pan Chicken Fajitas', 'Sheet Pan Fried Rice', 'Vegetarian Chinese Fried Noodles', "Papa Drexler's Bavarian Pretzels", 'Quick Bruschetta Chicken Bake', 'Spaghetti alla Carbonara: the Traditional Italian Recipe', 'Stir-Fry Chicken and Vegetables', 'Vegetarian Moussaka', 'French Baguettes', 'Shrimp Fettuccine Alfredo', 'Skillet Chicken Picante', 'Spaghetti Sauce', 'Vegetarian Korma', 'Fettuccini Carbonara', 'Kaese Spaetzle', 'Beef and Beet Borscht', 'Addictive Sweet Potato Burritos', "Chef John's French Omelette", 'Sweet and Spicy Stir Fry with Chicken and Broccoli', 'Simple Sweet and Spicy Chicken Wraps', 'Johnny Marzetti Casserole', 'Th

In [1129]:
fw_names

['Scandinavian Pickled Beets',
 'Sweet Tamale Pie',
 'Homemade Farfalle Pasta',
 'Pan de Muertos',
 'Spicy Italian Pork Cutlets',
 'Caldo Verde',
 'Zwetschgendatschi',
 'Rigatoni with Secret Sauce',
 'Homemade Muesli',
 'Indian Sweet Bread',
 'Spaetzle, Sauerkraut and Sausage Casserole',
 'Homemade Adobo All-Purpose Seasoning',
 'Homemade Italian Sausage',
 'Homemade Mince Pie with Crumbly Topping',
 'Brazilian Lemonade',
 'Vietnamese Dipping Sauce',
 'Kaese Spaetzle',
 'Birria de Res Tacos',
 'Vegetarian Pad Thai',
 'Keftedes - Greek Meatballs',
 'Homemade Feta Cheese',
 'Vegetarian Mulligatawny Soup',
 'Beef with Vegetables',
 'Garlic and Herb Marinade',
 'Lamb Feta Peppers',
 'Vegetable Curry',
 'Kotlet Schabowy',
 'Hot and Sweet Ginger Drink',
 'Creamy Chicken Marsala Fettuccine',
 'Spiced Sweet Roasted Red Pepper Hummus',
 'Shredded Beef Tacos with Lime',
 'Upside Down',
 'Vietnamese Beef Pho',
 'Cheesy Vegetable Lasagna',
 'Teriyaki Sauce and Marinade',
 'African Sweet Potato Ste

For the remaining names with bracket at the end, split into two new recipe names

In [1130]:
for i, recipe in enumerate(p_recipes):
    try:
        if p_recipes[i]["name"] in b_name_end:
            newname1, newname2 = break_fw_bracket(p_recipes[i]["name"])
            print(p_recipes[i]["name"])
            new_recipe1 = {'name': newname1, 'ingredients': p_recipes[i]["ingredients"]}
            new_recipe2 = {'name': newname2, 'ingredients': p_recipes[i]["ingredients"]}
            p_recipes.append(new_recipe1)
            p_recipes.append(new_recipe2)
            p_recipes.remove(p_recipes[i])
    except Exception as e:
        pass

tagged_recipe_names = retag(p_recipes, "name")

Tres Leches (Milk Cake)
Chiles en Nogada (Mexican Stuffed Poblano Peppers in Walnut Sauce)
Spicy Thai Basil Chicken (Pad Krapow Gai)
Labneh (Lebanese Yogurt)
Indian Chicken Curry (Murgh Kari)
Keema Aloo (Ground Beef and Potatoes)
Turkish Eggs (Cilbir)
South African Melktert (Milk Tart)
Ukrainian Apple Cake (Yabluchnyk)
Spanish Garlic Shrimp (Gambas al Ajillo)
German Potato Dumplings (Kartoffelkloesse)
Apfelkuchen (Apple Cake)
Eggplant Caponata (Sicilian Version)
Chana Masala (Savory Indian Chick Peas)
Ricotta Pie (Old Italian Recipe)
Easy Blini (Russian Pancake)
Easy Bulgogi (Korean BBQ Beef)
Carne en su Jugo (Meat in its Juices)
Ghormeh Sabzi (Persian Herb Stew)
Puerto Rican Tostones (Fried Plantains)
Kalbi (Korean BBQ Short Ribs)
Macaron (French Macaroon)
Atsara (Papaya Relish)
Authentic Chinese Egg Rolls ()
Greek Lentil Soup (Fakes)
Lumpia (Shanghai version)
Northern Ontario Partridge (Ruffed Grouse)
Vampiros Mexicanos (Mexican Vampires)
Jamaican Saltfish Fritters (Stamp and Go)
Slo

For some reasons, need to run the cell twice

In [1131]:
for i, recipe in enumerate(p_recipes):
    try:
        if p_recipes[i]["name"] in b_name_end:
            newname1, newname2 = break_fw_bracket(p_recipes[i]["name"])
            print(p_recipes[i]["name"])
            new_recipe1 = {'name': newname1, 'ingredients': p_recipes[i]["ingredients"]}
            new_recipe2 = {'name': newname2, 'ingredients': p_recipes[i]["ingredients"]}
            p_recipes.append(new_recipe1)
            p_recipes.append(new_recipe2)
            p_recipes.remove(p_recipes[i])
    except Exception as e:
        pass

tagged_recipe_names = retag(p_recipes, "name")

Polish Noodles (Cottage Cheese and Noodles)
Oyakodon (Japanese Chicken and Egg Rice Bowl)
Papas Rellenas (Fried Stuffed Potatoes)
Blaukraut (German Red Cabbage)
Irish Boiled Dinner (Corned Beef)
True Dominican Sancocho (Latin 7-Meat Stew)
Blini (Russian Pancakes)
Oeufs Cocotte (Baked Eggs)
Ropa Vieja (Cuban Beef)
Lace Cookies (Florentine Cookies)
Sinigang na Bangus (Filipino Milkfish in Tamarind Broth)
Schwabischer Kartoffelsalat (German Potato Salad - Schwabisch Style)
Roti Canai or Paratha (Indian Pancake)
Melanzana alla Parmigiana (Perfect Eggplant Parmigiana)
Pierogi (Traditional Polish Dumplings)
Nipples of Venus (Capezzoli di Venere)
Samosadilla (Samosa Quesadilla)
Bulgogi (Korean Barbecued Beef)
Sabaayad (Somali Flatbread)
Filipino Baked Milkfish (Baked Bangus)
Ash-e Reshteh (Persian Legume Soup)
Lentil and Cactus Soup (Mom's Recipe)
Ethiopian Cabbage and Potato Dish (Atkilt)
Finnish Kropser (Baked Pancakes)
Oma's Griessnockerlsuppe (Beef and Semolina Dumpling Soup)
Kewa Datshi 

Only the names with bracket in the middle of their names remain

In [1132]:
bracketed_names = []
for bracket in bracket_tokens:
    names = find_value_with_char(p_recipes, 'name', bracket)
    print(names)
    bracketed_names= bracketed_names + names

bracketed_names = list(set(bracketed_names))

['Classic Cuban Midnight (Medianoche) Sandwich', 'Spicy Indian (Gujarati) Green Beans', "World's Best () Lasagna", 'Karaage (Japanese Fried Chicken) with Honey Mayoster Sauce', 'Kimchi Jun (Kimchi Pancake) and Dipping Sauce', 'Bee Sting Cake (Bienenstich) II', 'Coconut (Haupia) and Chocolate Pie', 'Lamb (Gosht) Biryani', 'Jeera (Cumin) Rice', 'Pollo (Chicken) Fricassee from Puerto Rico', 'Fish Sinigang (Tilapia) - Filipino Sour Broth Dish', 'Lazy Golumpki (Stuffed Cabbage) Soup', 'Ulu (Breadfruit) Pancakes', 'Fried Chicken Chunks (Chicharrones De Pollo) Dominican', 'Fusilli with Rapini (Broccoli Rabe), Garlic, and Tomato Wine Sauce', 'Seaweed (Nori) Soup', 'Vareniki (Russian Pierogi) with Potatoes and Mushrooms', 'Hawaiian Bruddah Potato Mac (Macaroni) Salad', 'Korean Bean Curd (Miso) Soup', 'Lengua (Beef Tongue) Stew', 'Albondigas (Meatballs) en Chipotle', 'Zito (Zhito or Koljivo) - Serbian Wheat Pudding', 'Besan (Gram Flour) Halwa']


Mac and rapini is only synonymous the the one word before them. Otherwise, the bracketed words are synonymous to all the words before them combined.

In [1133]:
bracketed_names

['Fried Chicken Chunks (Chicharrones De Pollo) Dominican',
 "World's Best () Lasagna",
 'Kimchi Jun (Kimchi Pancake) and Dipping Sauce',
 'Karaage (Japanese Fried Chicken) with Honey Mayoster Sauce',
 'Fish Sinigang (Tilapia) - Filipino Sour Broth Dish',
 'Spicy Indian (Gujarati) Green Beans',
 'Lazy Golumpki (Stuffed Cabbage) Soup',
 'Lengua (Beef Tongue) Stew',
 'Jeera (Cumin) Rice',
 'Vareniki (Russian Pierogi) with Potatoes and Mushrooms',
 'Hawaiian Bruddah Potato Mac (Macaroni) Salad',
 'Coconut (Haupia) and Chocolate Pie',
 'Lamb (Gosht) Biryani',
 'Ulu (Breadfruit) Pancakes',
 'Fusilli with Rapini (Broccoli Rabe), Garlic, and Tomato Wine Sauce',
 'Albondigas (Meatballs) en Chipotle',
 'Pollo (Chicken) Fricassee from Puerto Rico',
 'Korean Bean Curd (Miso) Soup',
 'Besan (Gram Flour) Halwa',
 'Classic Cuban Midnight (Medianoche) Sandwich',
 'Bee Sting Cake (Bienenstich) II',
 'Seaweed (Nori) Soup',
 'Zito (Zhito or Koljivo) - Serbian Wheat Pudding']

The names can still be duplicated into 2, except that the bracketed word replaces the words before in the second new name, treating them as synonyms.

In [1134]:
def convert_bracket_synonym(name, num=0):
    name1 = re.findall(BRACKET_REGEX, name)[0]
    name1 = name1[name1.find("(")+1:name1.find(")")]
    name1_suffix = name.split(')')[1]
    if num==0:
        name1 = name1 + name1_suffix
        name2 = re.sub(BRACKET_REGEX, "", name)
    else:
        name1_prefix = name.split('(')[0]
        name1_prefix = name1_prefix[:-num]
        name1 = name1_prefix + name1 + name1_suffix
        name2 = re.sub(BRACKET_REGEX, " ", name)
    return name1, name2

print(convert_bracket_synonym("Lamb (Gosht) Biryani"))
print(convert_bracket_synonym("Fusilli with Rapini (Broccoli Rabe), Garlic, and Tomato Wine Sauce", 1))
print(convert_bracket_synonym("Hawaiian Bruddah Potato Mac (Macaroni) Salad", 1))

('Gosht Biryani', 'Lamb Biryani')
('Fusilli with RapiniBroccoli Rabe, Garlic, and Tomato Wine Sauce', 'Fusilli with Rapini , Garlic, and Tomato Wine Sauce')
('Hawaiian Bruddah Potato MacMacaroni Salad', 'Hawaiian Bruddah Potato Mac  Salad')


In [1135]:
for i, recipe in enumerate(p_recipes):
    try:
        if p_recipes[i]["name"] in b_name_mid:
            newname1, newname2 = convert_bracket_synonym(p_recipes[i]["name"])
            print(p_recipes[i]["name"])
            new_recipe1 = {'name': newname1, 'ingredients': p_recipes[i]["ingredients"]}
            new_recipe2 = {'name': newname2, 'ingredients': p_recipes[i]["ingredients"]}
            p_recipes.append(new_recipe1)
            p_recipes.append(new_recipe2)
            p_recipes.remove(p_recipes[i])
    except Exception as e:
        pass

tagged_recipe_names = retag(p_recipes, "name")

Classic Cuban Midnight (Medianoche) Sandwich
Spicy Indian (Gujarati) Green Beans
World's Best () Lasagna
Karaage (Japanese Fried Chicken) with Honey Mayoster Sauce
Kimchi Jun (Kimchi Pancake) and Dipping Sauce
Bee Sting Cake (Bienenstich) II
Coconut (Haupia) and Chocolate Pie
Lamb (Gosht) Biryani
Jeera (Cumin) Rice
Pollo (Chicken) Fricassee from Puerto Rico
Fish Sinigang (Tilapia) - Filipino Sour Broth Dish
Lazy Golumpki (Stuffed Cabbage) Soup
Ulu (Breadfruit) Pancakes
Fried Chicken Chunks (Chicharrones De Pollo) Dominican
Fusilli with Rapini (Broccoli Rabe), Garlic, and Tomato Wine Sauce
Seaweed (Nori) Soup
Vareniki (Russian Pierogi) with Potatoes and Mushrooms
Hawaiian Bruddah Potato Mac (Macaroni) Salad
Korean Bean Curd (Miso) Soup
Lengua (Beef Tongue) Stew
Albondigas (Meatballs) en Chipotle
Zito (Zhito or Koljivo) - Serbian Wheat Pudding
Besan (Gram Flour) Halwa


Successfully removed all brackets from recipe names

In [1136]:
bracketed_names = []
for bracket in bracket_tokens:
    names = find_value_with_char(p_recipes, 'name', bracket)
    print(names)
    bracketed_names= bracketed_names + names

bracketed_names = list(set(bracketed_names))
bracketed_names

[]


[]

Dashes are mostly adjectives, but things like semi colon need to be removed. As for colons, its mostly translation. Semicolons are caused by K&auml;, which are dishes with special characters or German words.

In [1137]:
colon_tokens = list(set(list_words_with_tag(tagged_recipe_names, ":")))
colon_tokens

[':', ';', '-']

In [1138]:
for colon in colon_tokens:
  print(find_value_with_char(p_recipes, 'name', colon))

['Spaghetti alla Carbonara: the Traditional Italian Recipe', 'Doro Wat: Ethiopian Chicken Dish', "Grandma's Focaccia: Baraise Style"]
['Quorn&trade; and Chickpea Curry', 'Empanadas Salte&ntilde;as', 'Sp&auml;tzle', 'Tamales Oaxaque&ntilde;os', 'K&auml;sesahnetorte', 'K&auml;sesahnetorte']
['Pan-Fried Asparagus', 'Super-Delicious Zuppa Toscana', 'Indian-Style Chicken and Onions', 'Haluski - Cabbage and Noodles', 'Chicken Stir-Fry', 'Quick Beef Stir-Fry', 'How to Make Coquilles Saint-Jacques', 'Mexican-Style Chicken Taco Casserole', 'Make-Ahead Vegetarian Moroccan Stew', 'Japanese-Style Deep-Fried Shrimp', 'Carnitas - Pressure Cooker', 'Chicken and Broccoli Stir-Fry', 'Broccoli and Chicken Stir-Fry', 'Ginger Veggie Stir-Fry', 'White Chicken Enchilada Slow-Cooker Casserole', 'Old-Fashioned Swedish Glogg', 'Stir-Fry Chicken and Vegetables', 'Barbacoa-Style Shredded Beef', 'Simple Slow-Cooked Korean Beef Soft Tacos', 'Air-Fried Korean Chicken Wings', 'Kouign-Amann', 'Gnocchi with Sage-Butte

In [1139]:
def remove_entry_with(dict_list, target, key="name"):
    for i, recipe in enumerate(dict_list):
        try:
            if target in dict_list[i]["name"]:
                dict_list.remove(dict_list[i])
        except Exception as e:
            pass

In [1140]:
for semicolon in ["Quorn&trade;", "Sp&auml;tzle", "Tamales Oaxaque&ntilde;os", "K&auml;sesahnetorte", "Salte&ntilde;as"]:
    remove_entry_with(p_recipes, semicolon)
tagged_recipe_names = retag(p_recipes, "name")

Semi colons cleaned

In [1141]:
colon_tokens = list(set(list_words_with_tag(tagged_recipe_names, ":")))
colon_tokens

[':', '-']

In [1142]:
for colon in colon_tokens:
  print(find_value_with_char(p_recipes, 'name', colon))

['Spaghetti alla Carbonara: the Traditional Italian Recipe', 'Doro Wat: Ethiopian Chicken Dish', "Grandma's Focaccia: Baraise Style"]
['Pan-Fried Asparagus', 'Super-Delicious Zuppa Toscana', 'Indian-Style Chicken and Onions', 'Haluski - Cabbage and Noodles', 'Chicken Stir-Fry', 'Quick Beef Stir-Fry', 'How to Make Coquilles Saint-Jacques', 'Mexican-Style Chicken Taco Casserole', 'Make-Ahead Vegetarian Moroccan Stew', 'Japanese-Style Deep-Fried Shrimp', 'Carnitas - Pressure Cooker', 'Chicken and Broccoli Stir-Fry', 'Broccoli and Chicken Stir-Fry', 'Ginger Veggie Stir-Fry', 'White Chicken Enchilada Slow-Cooker Casserole', 'Old-Fashioned Swedish Glogg', 'Stir-Fry Chicken and Vegetables', 'Barbacoa-Style Shredded Beef', 'Simple Slow-Cooked Korean Beef Soft Tacos', 'Air-Fried Korean Chicken Wings', 'Kouign-Amann', 'Gnocchi with Sage-Butter Sauce', 'Giant Bacon-Wrapped Meatballs', 'Low-Carb Cauliflower Rice Sushi Rolls', 'Onigiri - Japanese Rice Balls', "Frank's Favorite Slow-Cooker Thai Chic

For these 2 names, colons are used for describing

In [1143]:
# Spaghetti alla Carbonara: the Traditional Italian Recipe => traditional Italian Spaghetti alla Carbonara
searchReplaceAddPattList(p_recipes, r": the Traditional Italian Recipe", "", "traditional Italian ")
# Grandma's Focaccia: Baraise Style => Grandma's Baraise Style Focaccia
searchReplaceAddPattList(p_recipes, r": Baraise Style", "", "Baraise style ", index=10)
tagged_recipe_names = retag(p_recipes, "name")

Cleaned 2 names with colon. If the dashes are between a word, they are either part of a word's spelling or joining two words together, typically as an adjective. However, if it is between spaces, they are translations.

In [1144]:
colon_tokens = list(set(list_words_with_tag(tagged_recipe_names, ":")))
colon_tokens

[':', '-']

In [1145]:
new_colon_names = []
for colon in colon_tokens:
    print(find_value_with_char(p_recipes, 'name', colon))
    new_colon_names=new_colon_names+find_value_with_char(p_recipes, 'name', colon)
new_colon_names

['Doro Wat: Ethiopian Chicken Dish']
['Pan-Fried Asparagus', 'Super-Delicious Zuppa Toscana', 'Indian-Style Chicken and Onions', 'Haluski - Cabbage and Noodles', 'Chicken Stir-Fry', 'Quick Beef Stir-Fry', 'How to Make Coquilles Saint-Jacques', 'Mexican-Style Chicken Taco Casserole', 'Make-Ahead Vegetarian Moroccan Stew', 'Japanese-Style Deep-Fried Shrimp', 'Carnitas - Pressure Cooker', 'Chicken and Broccoli Stir-Fry', 'Broccoli and Chicken Stir-Fry', 'Ginger Veggie Stir-Fry', 'White Chicken Enchilada Slow-Cooker Casserole', 'Old-Fashioned Swedish Glogg', 'Stir-Fry Chicken and Vegetables', 'Barbacoa-Style Shredded Beef', 'Simple Slow-Cooked Korean Beef Soft Tacos', 'Air-Fried Korean Chicken Wings', 'Kouign-Amann', 'Gnocchi with Sage-Butter Sauce', 'Giant Bacon-Wrapped Meatballs', 'Low-Carb Cauliflower Rice Sushi Rolls', 'Onigiri - Japanese Rice Balls', "Frank's Favorite Slow-Cooker Thai Chicken", 'Two-Ingredient Naan', 'Chicken French - Rochester, NY Style', 'Velveting Chicken Breast, C

['Doro Wat: Ethiopian Chicken Dish',
 'Pan-Fried Asparagus',
 'Super-Delicious Zuppa Toscana',
 'Indian-Style Chicken and Onions',
 'Haluski - Cabbage and Noodles',
 'Chicken Stir-Fry',
 'Quick Beef Stir-Fry',
 'How to Make Coquilles Saint-Jacques',
 'Mexican-Style Chicken Taco Casserole',
 'Make-Ahead Vegetarian Moroccan Stew',
 'Japanese-Style Deep-Fried Shrimp',
 'Carnitas - Pressure Cooker',
 'Chicken and Broccoli Stir-Fry',
 'Broccoli and Chicken Stir-Fry',
 'Ginger Veggie Stir-Fry',
 'White Chicken Enchilada Slow-Cooker Casserole',
 'Old-Fashioned Swedish Glogg',
 'Stir-Fry Chicken and Vegetables',
 'Barbacoa-Style Shredded Beef',
 'Simple Slow-Cooked Korean Beef Soft Tacos',
 'Air-Fried Korean Chicken Wings',
 'Kouign-Amann',
 'Gnocchi with Sage-Butter Sauce',
 'Giant Bacon-Wrapped Meatballs',
 'Low-Carb Cauliflower Rice Sushi Rolls',
 'Onigiri - Japanese Rice Balls',
 "Frank's Favorite Slow-Cooker Thai Chicken",
 'Two-Ingredient Naan',
 'Chicken French - Rochester, NY Style',
 

But in some cases, they are words after the dashes describe the dish, such as Rochester, NY Style and Restaurant Style

In [1146]:
for colname in new_colon_names:
    if len(re.findall("( - )|(: )", colname)) > 0:
        print(colname)

Doro Wat: Ethiopian Chicken Dish
Haluski - Cabbage and Noodles
Carnitas - Pressure Cooker
Onigiri - Japanese Rice Balls
Chicken French - Rochester, NY Style
Taqueria Style Tacos - Carne Asada
Al Kabsa - Traditional Saudi Rice and Chicken
Italian Subs - Restaurant Style
Bazlama - Turkish Flat Bread
Norwegian Pancakes - Pannekaken
Pain de Campagne - Country French Bread
Flemish Frites - Belgian Fries with Andalouse Sauce
Portuguese Custard Tarts - Pasteis de Nata
Eggplant Parmesan - Gluten-Free
Tonkatsu - Asian-Style Pork Chop
Indian Eggplant - Bhurtha
Hot Pepper Sauce - A Trinidadian Staple
The Sarge's Goetta - German Breakfast Treat
Italian Sausage - Tuscan Style
Honey Milk Tea - Hong Kong Style
Mexican Lasagna - No Lasagna Noodles!
Lumpia - Filipino Shrimp and Pork Egg Rolls
Portuguese Muffins - Bolo Levedo
Curry Pasta - Pakistani Style
Cauliflower and Potato Stir-Fry - East Indian Recipe
Keftedes - Greek Meatballs
Brasato al Barolo - Braised Chuck Roast in Red Wine
Potato Salad - Ger

Replace or remove the remaining dashes that are surrounded by spaces

In [1147]:
# Chicken French - Rochester, NY Style => Rochester, NY Style Chicken French
searchReplaceAddPattList(p_recipes, r" - Rochester, NY Style", "", "Rochester, NY Style ")
# Carnitas - Pressure Cooker => pressure cooker carnitas
searchReplaceAddPattList(p_recipes, r" - Rochester, NY Style", "", "Rochester, NY Style ")
# Italian Subs - Restaurant Style => restaurant style Italian subs
searchReplaceAddPattList(p_recipes, r" - Restaurant Style", "", "restaurant style ")
# Eggplant Parmesan - Gluten-Free => glutten-free Eggplant Parmesan
searchReplaceAddPattList(p_recipes, r" - Gluten-Free", "", "glutten-free ")
# Italian Sausage - Tuscan Style => Tuscan style Italian Sausage
searchReplaceAddPattList(p_recipes, r" - Tuscan Style", "", "Tuscan style ")
# Honey Milk Tea - Hong Kong Style => Hong Kong style Honey Milk Tea
searchReplaceAddPattList(p_recipes, r" - Hong Kong Style", "", "Hong Kong style ")
# Curry Pasta - Pakistani Style => Pakistani style Curry Pasta
searchReplaceAddPattList(p_recipes, r" - Pakistani Style", "", "Pakistani style ")
# Cauliflower and Potato Stir-Fry - East Indian Recipe => East Indian style Cauliflower and Potato Stir-Fry
searchReplaceAddPattList(p_recipes, r" - East Indian Recipe", "", "East Indian style ")
# German Potato Salad - Schwabisch Style => Schwabisch style German Potato Salad
searchReplaceAddPattList(p_recipes, r" - Schwabisch Style", "", "Schwabisch style ")
# Tilapia - Filipino Sour Broth Dish => Filipino Sour Broth tilapia
searchReplaceAddPattList(p_recipes, r"Tilapia - ", "", "tilapia", index=20)
# Fish Sinigang - Filipino Sour Broth Dish - Schwabisch Style => Filipino Sour Broth Sinigang fish
searchReplaceAddPattList(p_recipes, r"Fish Sinigang - ", "", "Sinigang fish", index=20)

# remove  - A Trinidadian Staple from Hot Pepper Sauce - A Trinidadian Staple
searchReplacePattList(p_recipes, r" - A Trinidadian Staple", "")
# remove  - German Breakfast Treat from The Sarge's Goetta - German Breakfast Treat
searchReplacePattList(p_recipes, r" - German Breakfast Treat", "")
# remove  - No Lasagna Noodles! from Mexican Lasagna - No Lasagna Noodles!
searchReplacePattList(p_recipes, r" - No Lasagna Noodles!", "")
# remove  - Not Just for Chicken from Sweet and Sour Jam - Not Just for Chicken
searchReplacePattList(p_recipes, r" - Not Just for Chicken", "")
                      
tagged_recipe_names = retag(p_recipes, "name")

In [1148]:
new_colon_names = []
for colon in colon_tokens:
    print(find_value_with_char(p_recipes, 'name', colon))
    new_colon_names=new_colon_names+find_value_with_char(p_recipes, 'name', colon)
new_colon_names

['Doro Wat: Ethiopian Chicken Dish']
['Pan-Fried Asparagus', 'Super-Delicious Zuppa Toscana', 'Indian-Style Chicken and Onions', 'Haluski - Cabbage and Noodles', 'Chicken Stir-Fry', 'Quick Beef Stir-Fry', 'How to Make Coquilles Saint-Jacques', 'Mexican-Style Chicken Taco Casserole', 'Make-Ahead Vegetarian Moroccan Stew', 'Japanese-Style Deep-Fried Shrimp', 'Carnitas - Pressure Cooker', 'Chicken and Broccoli Stir-Fry', 'Broccoli and Chicken Stir-Fry', 'Ginger Veggie Stir-Fry', 'White Chicken Enchilada Slow-Cooker Casserole', 'Old-Fashioned Swedish Glogg', 'Stir-Fry Chicken and Vegetables', 'Barbacoa-Style Shredded Beef', 'Simple Slow-Cooked Korean Beef Soft Tacos', 'Air-Fried Korean Chicken Wings', 'Kouign-Amann', 'Gnocchi with Sage-Butter Sauce', 'Giant Bacon-Wrapped Meatballs', 'Low-Carb Cauliflower Rice Sushi Rolls', 'Onigiri - Japanese Rice Balls', "Frank's Favorite Slow-Cooker Thai Chicken", 'Two-Ingredient Naan', 'Velveting Chicken Breast, Chinese Restaurant-Style', 'Garlic-Herb L

['Doro Wat: Ethiopian Chicken Dish',
 'Pan-Fried Asparagus',
 'Super-Delicious Zuppa Toscana',
 'Indian-Style Chicken and Onions',
 'Haluski - Cabbage and Noodles',
 'Chicken Stir-Fry',
 'Quick Beef Stir-Fry',
 'How to Make Coquilles Saint-Jacques',
 'Mexican-Style Chicken Taco Casserole',
 'Make-Ahead Vegetarian Moroccan Stew',
 'Japanese-Style Deep-Fried Shrimp',
 'Carnitas - Pressure Cooker',
 'Chicken and Broccoli Stir-Fry',
 'Broccoli and Chicken Stir-Fry',
 'Ginger Veggie Stir-Fry',
 'White Chicken Enchilada Slow-Cooker Casserole',
 'Old-Fashioned Swedish Glogg',
 'Stir-Fry Chicken and Vegetables',
 'Barbacoa-Style Shredded Beef',
 'Simple Slow-Cooked Korean Beef Soft Tacos',
 'Air-Fried Korean Chicken Wings',
 'Kouign-Amann',
 'Gnocchi with Sage-Butter Sauce',
 'Giant Bacon-Wrapped Meatballs',
 'Low-Carb Cauliflower Rice Sushi Rolls',
 'Onigiri - Japanese Rice Balls',
 "Frank's Favorite Slow-Cooker Thai Chicken",
 'Two-Ingredient Naan',
 'Velveting Chicken Breast, Chinese Restau

The remaining names with dashes surrounded by dashes are translations, which can be split into two names

In [1149]:
colnames_to_split = []
for colname in new_colon_names:
    if len(re.findall("( - )|(: )", colname)) > 0:
        print(colname)
        colnames_to_split.append(colname)

Doro Wat: Ethiopian Chicken Dish
Haluski - Cabbage and Noodles
Carnitas - Pressure Cooker
Onigiri - Japanese Rice Balls
Taqueria Style Tacos - Carne Asada
Al Kabsa - Traditional Saudi Rice and Chicken
Bazlama - Turkish Flat Bread
Norwegian Pancakes - Pannekaken
Pain de Campagne - Country French Bread
Flemish Frites - Belgian Fries with Andalouse Sauce
Portuguese Custard Tarts - Pasteis de Nata
Tonkatsu - Asian-Style Pork Chop
Indian Eggplant - Bhurtha
Lumpia - Filipino Shrimp and Pork Egg Rolls
Portuguese Muffins - Bolo Levedo
Keftedes - Greek Meatballs
Brasato al Barolo - Braised Chuck Roast in Red Wine
Potato Salad - German Kartoffel
Tembleque de Coco - Coconut Tembleque
Kroppkakor - Swedish Potato Dumplings
Ladolemono - Lemon Oil Sauce for Fish or Chicken
Mie Goreng - Indonesian Fried Noodles
Vaselopita - Greek New Years Cake
Knedliky - Czech Dumpling with Sauerkraut
Zhito or Koljivo - Serbian Wheat Pudding
Zito - Serbian Wheat Pudding


In [1150]:
for i, recipe in enumerate(p_recipes):
    try:
        if p_recipes[i]["name"] in colnames_to_split:
            splits = re.split("( - )|(: )", p_recipes[i]["name"])
            newname1 = splits[0]
            newname2 = splits[len(splits)-1]
            new_recipe1 = {'name': newname1, 'ingredients': p_recipes[i]["ingredients"]}
            new_recipe2 = {'name': newname2, 'ingredients': p_recipes[i]["ingredients"]}
            p_recipes.append(new_recipe1)
            p_recipes.append(new_recipe2)
            p_recipes.remove(p_recipes[i])
    except Exception as e:
        pass

tagged_recipe_names = retag(p_recipes, "name")

The remaining names with dash are those in words

In [1151]:
colon_tokens = list(set(list_words_with_tag(tagged_recipe_names, ":")))
colon_tokens

['-']

In [1152]:
new_colon_names = []
for colon in colon_tokens:
    print(find_value_with_char(p_recipes, 'name', colon))
    new_colon_names=new_colon_names+find_value_with_char(p_recipes, 'name', colon)
new_colon_names

['Pan-Fried Asparagus', 'Super-Delicious Zuppa Toscana', 'Indian-Style Chicken and Onions', 'Chicken Stir-Fry', 'Quick Beef Stir-Fry', 'How to Make Coquilles Saint-Jacques', 'Mexican-Style Chicken Taco Casserole', 'Make-Ahead Vegetarian Moroccan Stew', 'Japanese-Style Deep-Fried Shrimp', 'Chicken and Broccoli Stir-Fry', 'Broccoli and Chicken Stir-Fry', 'Ginger Veggie Stir-Fry', 'White Chicken Enchilada Slow-Cooker Casserole', 'Old-Fashioned Swedish Glogg', 'Stir-Fry Chicken and Vegetables', 'Barbacoa-Style Shredded Beef', 'Simple Slow-Cooked Korean Beef Soft Tacos', 'Air-Fried Korean Chicken Wings', 'Kouign-Amann', 'Gnocchi with Sage-Butter Sauce', 'Giant Bacon-Wrapped Meatballs', 'Low-Carb Cauliflower Rice Sushi Rolls', "Frank's Favorite Slow-Cooker Thai Chicken", 'Two-Ingredient Naan', 'Velveting Chicken Breast, Chinese Restaurant-Style', 'Garlic-Herb Linguine', 'Korean-style Seaweed Soup', 'Ube-Macapuno Cake', 'Cuban-Style Yuca', 'Japanese-Style Cabbage Salad', "Jorge's Indian-Spice

['Pan-Fried Asparagus',
 'Super-Delicious Zuppa Toscana',
 'Indian-Style Chicken and Onions',
 'Chicken Stir-Fry',
 'Quick Beef Stir-Fry',
 'How to Make Coquilles Saint-Jacques',
 'Mexican-Style Chicken Taco Casserole',
 'Make-Ahead Vegetarian Moroccan Stew',
 'Japanese-Style Deep-Fried Shrimp',
 'Chicken and Broccoli Stir-Fry',
 'Broccoli and Chicken Stir-Fry',
 'Ginger Veggie Stir-Fry',
 'White Chicken Enchilada Slow-Cooker Casserole',
 'Old-Fashioned Swedish Glogg',
 'Stir-Fry Chicken and Vegetables',
 'Barbacoa-Style Shredded Beef',
 'Simple Slow-Cooked Korean Beef Soft Tacos',
 'Air-Fried Korean Chicken Wings',
 'Kouign-Amann',
 'Gnocchi with Sage-Butter Sauce',
 'Giant Bacon-Wrapped Meatballs',
 'Low-Carb Cauliflower Rice Sushi Rolls',
 "Frank's Favorite Slow-Cooker Thai Chicken",
 'Two-Ingredient Naan',
 'Velveting Chicken Breast, Chinese Restaurant-Style',
 'Garlic-Herb Linguine',
 'Korean-style Seaweed Soup',
 'Ube-Macapuno Cake',
 'Cuban-Style Yuca',
 'Japanese-Style Cabbage 

!, ? and . are found, which are odd for recipe names

In [1153]:
punc_tokens = list_words_with_tag(tagged_recipe_names, ".")
punc_tokens

['!', '!', '!', '!', '.', '?']

The punctuations are mostly slang abbreviations and exclamations

In [1154]:
for punc in list(set(punc_tokens)):
  print(find_value_with_char(p_recipes, 'name', punc))

['Real Canadian Butter Tarts, eh?']
['Sangria! Sangria!', 'Oatmeal Apple Crisp To Die For!', "Sushi House Salad Dressing, It's ORANGE!"]
["Our Top P.F. Chang's Copycat Recipes", "Perfect St. Patrick's Day Cake"]


Remove the exclamations

In [1155]:
searchReplacePattList(p_recipes, r"! Sangria!", "")
searchReplacePattList(p_recipes, r" To Die For!", "")
searchReplacePattList(p_recipes, r", It's ORANGE!", "")
searchReplacePattList(p_recipes, r", eh\?", "")
searchReplacePattList(p_recipes, r"Our Top ", "")

tagged_recipe_names = retag(p_recipes, "name")

Fullstops that remain are part of recipe names

In [1156]:
punc_tokens = list_words_with_tag(tagged_recipe_names, ".")
punc_tokens

['.']

In [1157]:
for punc in list(set(punc_tokens)):
  print(find_value_with_char(p_recipes, 'name', punc))

["P.F. Chang's Copycat Recipes", "Perfect St. Patrick's Day Cake"]


Some 'that' can be found

In [1158]:
wdt_tokens = list_words_with_tag(tagged_recipe_names, "WDT")
wdt_tokens

['That', 'That', 'That', 'That']

The 'that's are used to add details, but not actual recipe name

In [1159]:
for wdt in list(set(wdt_tokens)):
  print(find_value_with_char(p_recipes, 'name', wdt))

['German Recipes That Are Comfort Food Favorites', 'Mexican-Inspired Ground Beef Casseroles That Deliver Big Flavor With Every Satisfying Bite', 'Tuscan Recipes That Reveal the Best of Italian Cooking', 'Easy Dinners That Start with Packaged Gnocchi', "That's-a Meatloaf", 'Favorite Recipes That Show Off Armenian Cuisine', 'Our Best Stir-Fry Recipes That Are Even Better Than Take-Out', 'Comforting Polish Cabbage Recipes That Are Family Favorites']


Remove

In [1160]:
searchReplacePattList(p_recipes, r" That Are Comfort Food Favorites", "")
searchReplacePattList(p_recipes, r" That Deliver Big Flavor With Every Satisfying Bite", "")
searchReplacePattList(p_recipes, r" That Reveal the Best of Italian Cooking", "")
searchReplacePattList(p_recipes, r"That's-a ", "")
searchReplacePattList(p_recipes, r"Favorite Recipes That Show Off ", "")
searchReplacePattList(p_recipes, r" That Are Even Better Than Take-Out", "")
searchReplacePattList(p_recipes, r" That Are Family Favorites", "")

searchReplaceAddPattList(p_recipes, r" That Start with Packaged Gnocchi", "", "packaged gnocchi ", index=5)
tagged_recipe_names = retag(p_recipes, "name")

That removed

In [1161]:
wdt_tokens = list_words_with_tag(tagged_recipe_names, "WDT")
wdt_tokens

[]

There's some 'how's

In [1162]:
wrb_tokens = list_words_with_tag(tagged_recipe_names, "WRB")
wrb_tokens

['How', 'How', 'How', 'How', 'How', 'How', 'How']

In [1163]:
for wrb in list(set(wrb_tokens)):
  print(find_value_with_char(p_recipes, 'name', wrb))

['How to Make Coquilles Saint-Jacques', 'How to Make Bolognese Sauce', 'How to Make Beef Satay', 'How to Make Peanut Dipping Sauce', 'How to Make Tres Leches Cake', 'How to Make Cassoulet', 'How to Make Turkey Manicotti']


Remove the 'how's and keep only the name

In [1164]:
searchReplacePattList(p_recipes, r"How to Make ", "")

tagged_recipe_names = retag(p_recipes, "name")

In [1165]:
list_words_with_tag(tagged_recipe_names, "WRB")

[]

There's some personal pronouns (possessive)

In [1166]:
prp_tokens = list_words_with_tag(tagged_recipe_names, "PRP$")
prp_tokens

['Our',
 'My',
 'My',
 'My',
 'Our',
 'My',
 'Our',
 'My',
 'My',
 'My',
 'Our',
 'My',
 'My',
 'Your',
 'Our',
 'Our',
 'Our',
 'My',
 'its']

In [1167]:
for prp in list(set(prp_tokens)):
  print(find_value_with_char(p_recipes, 'name', prp))

['Our Best Avgolemono Soup Recipes', 'Our Best Authentic Mexican Recipes', 'Our Best Empanada Recipes', 'Our Best Indian Recipes for Beginner Cooks', 'Our Best Stir-Fry Recipes', 'Our Favorite German Potato Recipes', 'Say Aloha to Our Best Hawaiian Recipes']
['Sweet Recipes to Complete Your Indian Dinner', 'Melt-in-Your-Mouth Beef Cacciatore', 'Polish Recipes to Make Your Grandmother Proud']
['My Own Famous Stuffed Grape Leaves', 'My Best Chicken Piccata', 'My Favorite Sesame Noodles', 'My Chicken Parmesan', "My Mom's Greek Lemon Rice", 'My Fly Stir-Fry', 'My Chicken Pho Recipe', 'My Tangy German Potato Salad', 'My Big Fat Greek Baked Beans', "My Grandmother's French Dressing"]
['Anzac Biscuits I', "Sadie's Buttermilk Biscuits", 'Canadian Tea Biscuits', 'Empire Biscuits', 'Pastitsio IV', 'Crescent Butter Biscuits', 'Pastitsio', "Nanny's Newfoundland Tea Biscuits", 'Meat in its Juices']


Most can be removed

In [1168]:
searchReplacePattList(p_recipes, r"Our ", "")
searchReplacePattList(p_recipes, r"Your ", "")
searchReplacePattList(p_recipes, r"Melt-in-Your-Mouth ", "")
searchReplacePattList(p_recipes, r"My Own ", "")
searchReplacePattList(p_recipes, r"My Best ", "")
searchReplacePattList(p_recipes, r"My Favorite ", "")
searchReplacePattList(p_recipes, r"My Mom's ", "")
searchReplacePattList(p_recipes, r"My Grandmother's ", "")
searchReplacePattList(p_recipes, r"My ", "")

tagged_recipe_names = retag(p_recipes, "name")

The remaining ones are misclassified tags by nltk

In [1169]:
prp_tokens = list_words_with_tag(tagged_recipe_names, "PRP$")
prp_tokens

['its']

In [1170]:
for prp in list(set(prp_tokens)):
  print(find_value_with_char(p_recipes, 'name', prp))

['Anzac Biscuits I', "Sadie's Buttermilk Biscuits", 'Canadian Tea Biscuits', 'Empire Biscuits', 'Pastitsio IV', 'Crescent Butter Biscuits', 'Pastitsio', "Nanny's Newfoundland Tea Biscuits", 'Meat in its Juices']


There's some personal pronouns

In [1171]:
prp_tokens = list_words_with_tag(tagged_recipe_names, "PRP")
prp_tokens

['I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'You',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'You',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'We',
 'I',
 'I',
 'I']

In [1172]:
for prp in list(set(prp_tokens)):
  print(find_value_with_char(p_recipes, 'name', prp))

['Shrimp Egg Foo Young', 'Good for You Greek Salad', 'Egg Foo Young', 'Quick and Easy Chinese Dinners You Can Make at Home', 'Young Coconut Jelly', 'Keto Egg Foo Young']
['German Apple Cake I', 'Indian-Style Chicken and Onions', 'Tender Italian Baked Chicken', 'Mexican Rice II', 'Sweet and Sour Chicken I', 'Chicken Cordon Bleu II', 'Hot German Potato Salad III', 'Stuffed Shells I', 'Baked Penne with Italian Sausage', 'German Pancakes II', "Grandma's Noodles II", 'Hot Italian Giardiniera', 'Russian Tea Cakes I', 'Fried Irish Cabbage with Bacon', 'Greek Salad I', 'Irish Boxty', 'Italian Sausage Stuffed Shells', 'Vegetarian Mexican Inspired Stuffed Peppers', 'Indian Saag', 'Chicken Enchiladas II', 'Chinese Chicken Fried Rice II', 'Quiche Lorraine I', 'Italian Cream Cheese and Ricotta Cheesecake', 'Taco Seasoning II', "Ingrid's Rouladen", 'Baked Italian Chicken Thighs', 'Original Homemade Italian Beef', 'Pork Chops Italiano', 'Pizzelles III', 'Eclairs II', 'Real Italian Calzones', 'Old Ita

Not much to remove, since most are misclassified POS

In [1173]:
searchReplacePattList(p_recipes, r" You Can Make at Home", "")

tagged_recipe_names = retag(p_recipes, "name")

Some base verbs can be removed

In [1174]:
vb_tokens = list_words_with_tag(tagged_recipe_names, "VB")
vb_tokens

['Take',
 'Make',
 'Take',
 'Kedgeree',
 'Swordfish',
 'Serve',
 'Make',
 'Celebrate',
 'Chicken',
 'Pata',
 'aux',
 'Poulet',
 'Papa',
 'Tarte',
 'Pollo',
 'Pancake',
 'Dutch',
 'Kransekake',
 'Dish',
 'Pannekaken']

In [1175]:
for vb in list(set(vb_tokens)):
  print(find_value_with_char(p_recipes, 'name', vb))

['Apple Tarte Tatin', 'Pear Tarte Tatin', 'Tarte de Pommes a la Normande', 'Tarte aux Moutarde', "Tarte a l'Oignon", 'Tarte Flambee']
['Dishes to Serve With Chapati']
["Chef John's Patatas Bravas", 'Authentic Patatas Bravas', 'Paksiw na Pata', 'Tajine de Poulet aux Carottes et Patates Douces']
['Spicy Korean Fried Chicken with Gochujang Sauce', 'Greek Lemon Chicken and Potato Bake', "Chef John's Chicken Kiev", 'Indian-Style Chicken and Onions', 'Tender Italian Baked Chicken', 'Chicken Katsu', 'Chicken Stir-Fry', 'Mexican-Style Chicken Taco Casserole', 'Curry Stand Chicken Tikka Masala Sauce', 'Chicken Enchiladas V', 'Jamaican Style Curry Chicken', 'Salsa Chicken', 'Grilled Asian Chicken', 'Chicken Tikka Masala', 'Sweet and Sour Chicken I', 'Chicken Cordon Bleu II', 'Turkish Chicken Kebabs', 'Chicken Souvlaki with Tzatziki Sauce', 'Greek Lemon Chicken Soup', 'Chicken Cacciatore in a Slow Cooker', 'Chicken and Broccoli Stir-Fry', 'Creamy Chicken Lasagna', 'Broccoli and Chicken Stir-Fry',

Remove recipe names with instruction

In [1176]:
searchReplacePattList(p_recipes, r" to Make at Home", "")
searchReplacePattList(p_recipes, r" to Make Grandmother Proud", "")
searchReplacePattList(p_recipes, r"Ways The World Makes Chicken And ", "")

searchReplaceAddPattList(p_recipes, r"Make Ahead ", "", "packaged gnocchi ")

tagged_recipe_names = retag(p_recipes, "name")

In [1177]:
for vb in list(set(vb_tokens)):
  print(find_value_with_char(p_recipes, 'name', vb))

['Apple Tarte Tatin', 'Pear Tarte Tatin', 'Tarte de Pommes a la Normande', 'Tarte aux Moutarde', "Tarte a l'Oignon", 'Tarte Flambee']
['Dishes to Serve With Chapati']
["Chef John's Patatas Bravas", 'Authentic Patatas Bravas', 'Paksiw na Pata', 'Tajine de Poulet aux Carottes et Patates Douces']
['Spicy Korean Fried Chicken with Gochujang Sauce', 'Greek Lemon Chicken and Potato Bake', "Chef John's Chicken Kiev", 'Indian-Style Chicken and Onions', 'Tender Italian Baked Chicken', 'Chicken Katsu', 'Chicken Stir-Fry', 'Mexican-Style Chicken Taco Casserole', 'Curry Stand Chicken Tikka Masala Sauce', 'Chicken Enchiladas V', 'Jamaican Style Curry Chicken', 'Salsa Chicken', 'Grilled Asian Chicken', 'Chicken Tikka Masala', 'Sweet and Sour Chicken I', 'Chicken Cordon Bleu II', 'Turkish Chicken Kebabs', 'Chicken Souvlaki with Tzatziki Sauce', 'Greek Lemon Chicken Soup', 'Chicken Cacciatore in a Slow Cooker', 'Chicken and Broccoli Stir-Fry', 'Creamy Chicken Lasagna', 'Broccoli and Chicken Stir-Fry',

Words like best and most can be removed

In [1178]:
rbs_tokens = list_words_with_tag(tagged_recipe_names, "RBS")
rbs_tokens

['Best', 'Most', 'Best']

In [1179]:
for rbs in list(set(rbs_tokens)):
  print(find_value_with_char(p_recipes, 'name', rbs))

['The Most Iconic French Desserts', 'Alfredo Mostaccioli']
['Best Bobotie', 'Best Fried Walleye', 'Best Avgolemono Soup Recipes', "Chef John's Best German Recipes", 'The Best Thai Peanut Sauce', 'Best Ever Russian Beef Stroganoff', "Grandma's Best Ever Sour Cream Lasagna", 'Best Guacamole', 'Best Ever Slow Cooker Italian Beef Roast', 'The Best Pavlova', "Savannah's Best Marinated Portobello Mushrooms", 'Best Peanut Sauce', 'Best Ever Carne Asada Marinade', "Mom's Best Spaghetti Sauce", 'The Best Korean Chicken Recipes', 'Best Instant Pot Chicken Cacciatore', 'Best Ziti Ever', 'Best Authentic Mexican Recipes', 'Best Empanada Recipes', 'Best Ziti Ever with Sausage', 'Best Chicken Parmesan', 'Best Pernil Ever', 'The Best Ricotta Pancakes', 'Best Indian Recipes for Beginner Cooks', 'Best Hot Sauce', 'Best Ever Irish Soda Bread', 'Best Hummus', 'The Best Thai Tom Kha Soup Recipe', 'Best French Macarons', 'Best Falafel', "Gordo's Best of the Best Lasagna", 'The Best Classic Beef Stroganoff',

In [1180]:
searchReplacePattList(p_recipes, r"Grandma's Best Ever ", "")
searchReplacePattList(p_recipes, r"Best Ever ", "")
searchReplacePattList(p_recipes, r"Best ", "")
searchReplacePattList(p_recipes, r" ever", "")
searchReplacePattList(p_recipes, r"The Most Iconic ", "")

tagged_recipe_names = retag(p_recipes, "name")

In [1181]:
rbs_tokens = list_words_with_tag(tagged_recipe_names, "RBS")
rbs_tokens

[]

Adverbs with -ly can be removed, except for the misclassified ones mainly caused by foreign recipe names

In [1182]:
rb_tokens = list_words_with_tag(tagged_recipe_names, "RB")
rb_tokens

['Absolutely',
 'Aebleskiver',
 'Incredibly',
 'Perfectly',
 'Absolutely',
 'Oven',
 'Perfectly',
 'Absolutely',
 'Heavenly',
 'Asiago',
 'Philly',
 'Family',
 'Deadly',
 'Yet',
 'Absolutely',
 'Ever',
 'Tourtiere',
 'Tourtiere',
 'Soon',
 'Here',
 'Long',
 'Tourtiere',
 'Tourtiere']

In [1183]:
for rb in list(set(rb_tokens)):
  print(find_value_with_char(p_recipes, 'name', rb))

['Deadly Delicious Lasagna']
['Ziti Ever', 'Ziti Ever with Sausage', 'Pernil Ever', 'Date Squares Ever']
['Perfectly Moist Irish Wheaten Bread', 'Perfectly Dry Roasted Chickpeas']
['No Tomato Paste Here']
['French Canadian Tourtiere', 'Traditional French Canadian Tourtiere', 'Reveillon Tourtiere', 'Tourtiere Spices', 'Tourtiere', 'Tourtiere', 'Tourtiere', 'Tourtiere']
['Chicken Long Rice Soup', 'Vietnamese Chicken and Long-Grain Rice Congee', 'Long Soup', 'Philippine Longanisa de Eugenio', 'Long Drink']
['Air Fryer Oven Taco Shells', 'Oven Kalua Pork', 'Oven-Roasted Chicken Thighs', 'Oven Baked Chicken Teriyaki', 'Oven-Baked Chicken Fajitas', 'Oven-Baked Teriyaki Chicken Thighs', 'Crispy Oven Beef-and-Bean Tostadas', "Oven-Roasted Za'atar Chicken Breasts", 'Chicken Chimi in the Oven']
['Heavenly Raspberry Dessert']
['Incredibly Delicious Italian Cream Cake']
['Asiago Sun-Dried Tomato Pasta', 'Chicken and Bowtie Pasta with Asiago Cream Sauce']
['Willard Family German Chocolate Cake', 'M

In [1184]:
searchReplacePattList(p_recipes, r"Deadly Delicious ", "")
searchReplacePattList(p_recipes, r"Heavenly ", "")
searchReplacePattList(p_recipes, r"Perfectly ", "")
searchReplacePattList(p_recipes, r"Absolutely Fabulous ", "")
searchReplacePattList(p_recipes, r"Absolutely Amazing  ", "")
searchReplacePattList(p_recipes, r"Absolutely Delicious ", "")
searchReplacePattList(p_recipes, r"Absolutely Perfect ", "")

searchReplaceAddPattList(p_recipes, r"No Tomato Paste Here", "", "tomato paste")

tagged_recipe_names = retag(p_recipes, "name")

In [1185]:
rb_tokens = list_words_with_tag(tagged_recipe_names, "RB")
rb_tokens

['Aebleskiver',
 'Incredibly',
 'Absolutely',
 'Oven',
 'Asiago',
 'Philly',
 'Family',
 'Yet',
 'Ever',
 'Tourtiere',
 'Tourtiere',
 'Soon',
 'Long',
 'Tourtiere',
 'Tourtiere']

In [1186]:
for rb in list(set(rb_tokens)):
  print(find_value_with_char(p_recipes, 'name', rb))

['Ziti Ever', 'Ziti Ever with Sausage', 'Pernil Ever', 'Date Squares Ever']
['Chicken Long Rice Soup', 'Vietnamese Chicken and Long-Grain Rice Congee', 'Long Soup', 'Philippine Longanisa de Eugenio', 'Long Drink']
['French Canadian Tourtiere', 'Traditional French Canadian Tourtiere', 'Reveillon Tourtiere', 'Tourtiere Spices', 'Tourtiere', 'Tourtiere', 'Tourtiere', 'Tourtiere']
['Air Fryer Oven Taco Shells', 'Oven Kalua Pork', 'Oven-Roasted Chicken Thighs', 'Oven Baked Chicken Teriyaki', 'Oven-Baked Chicken Fajitas', 'Oven-Baked Teriyaki Chicken Thighs', 'Crispy Oven Beef-and-Bean Tostadas', "Oven-Roasted Za'atar Chicken Breasts", 'Chicken Chimi in the Oven']
['Incredibly Delicious Italian Cream Cake']
['Asiago Sun-Dried Tomato Pasta', 'Chicken and Bowtie Pasta with Asiago Cream Sauce']
['Willard Family German Chocolate Cake', 'Mexican-Inspired Casseroles for Family-Pleasing Dinners', 'Chinese Happy Family', 'Family Sicilian Sauce and Meatballs', 'Greek Ground Beef Recipes Sure To Becom

In [1187]:
all_name_tags = []

for POS in ALL_POS:
  new_dic = {POS: list_words_with_tag(tagged_recipe_names, POS)}
  all_name_tags.append(new_dic)

get_tag_number(all_name_tags)

[{'$': 1},
 {"''": 7},
 {'(': 0},
 {')': 0},
 {',': 62},
 {'--': 0},
 {'.': 1},
 {':': 1},
 {'CC': 506},
 {'CD': 23},
 {'DT': 96},
 {'EX': 0},
 {'FW': 67},
 {'IN': 464},
 {'JJ': 1898},
 {'JJR': 2},
 {'JJS': 1},
 {'LS': 0},
 {'MD': 0},
 {'NN': 659},
 {'NNP': 12710},
 {'NNPS': 36},
 {'NNS': 389},
 {'PDT': 0},
 {'POS': 345},
 {'PRP': 69},
 {'PRP$': 1},
 {'RB': 15},
 {'RBR': 0},
 {'RBS': 0},
 {'RP': 2},
 {'SYM': 0},
 {'TO': 10},
 {'UH': 0},
 {'VB': 18},
 {'VBD': 39},
 {'VBG': 59},
 {'VBN': 139},
 {'VBP': 9},
 {'VBZ': 29},
 {'WDT': 0},
 {'WP': 0},
 {'WP$': 0},
 {'WRB': 0},
 {'``': 6}]

## Examining other POS in names

In [1188]:
vbz_tokens = list_words_with_tag(tagged_recipe_names, "VBZ")
vbz_tokens

['Ties',
 'el',
 'Leaves',
 'al',
 'al',
 'Leaves',
 'au',
 'di',
 'Ways',
 'de',
 'al',
 'Breasts',
 'en',
 'e',
 'al',
 'Leaves',
 'Breasts',
 'Squares',
 'al',
 'di',
 'aux',
 'di',
 'Leaves',
 'au',
 'di',
 'di',
 'al',
 'en',
 'en']

In [1189]:
vbp_tokens = list_words_with_tag(tagged_recipe_names, "VBP")
vbp_tokens

['Rellenos',
 'Greek',
 'Divine',
 'Wat',
 'Be',
 'en',
 'Mexicanos',
 'Rellenos',
 'en']

In [1190]:
vbg_tokens = list_words_with_tag(tagged_recipe_names, "VBG")
vbg_tokens

['Seasoning',
 'Dressing',
 'Pudding',
 'Using',
 'Canning',
 'Pudding',
 'Velveting',
 'Pudding',
 'Pudding',
 'Pudding',
 'Seasoning',
 'Comforting',
 'Seasoning',
 'Pouding',
 'Pudding',
 'Amazing',
 'Pudding',
 'Refreshing',
 'Pudding',
 'Seasoning',
 'Dressing',
 'Comforting',
 'Pudding',
 'Making',
 'Comforting',
 'Pudding',
 'Dumpling',
 'Dipping',
 'Refreshing',
 'Pudding',
 'Seasoning',
 'Seasoning',
 'Filling',
 'Thanksgiving',
 'Stuffing',
 'Pudding',
 'Pudding',
 'Refreshing',
 'Pudding',
 'Sizzling',
 'Topping',
 'Amazing',
 'Refreshing',
 'Comforting',
 'Dressing',
 'Using',
 'Seasoning',
 'Refreshing',
 'Pudding',
 'Pudding',
 'Pudding',
 'Ping',
 'Pudding',
 'Pudding',
 'Pudding',
 'Pudding',
 'Pudding',
 'Dumpling',
 'Pudding']

In [1191]:
vbd_tokens = list_words_with_tag(tagged_recipe_names, "VBD")
vbd_tokens

['Braised',
 'Corned',
 'Corned',
 'Pickled',
 'Shredded',
 'Braised',
 'Fashioned',
 'Filled',
 'Corned',
 'Fashioned',
 'Pickled',
 'Braised',
 'Breaded',
 'Fried',
 'Grilled',
 'Braised',
 'Pickled',
 'Braised',
 'Braised',
 'Planked',
 'Corned',
 'Corned',
 'Braised',
 'Infused',
 'Corned',
 'Obsessed',
 'Pickled',
 'Pulled',
 'Roasted',
 'Broiled',
 'Pickled',
 'Roasted',
 'di',
 'Braised',
 'Braised',
 'Pickled',
 'Mulled',
 'Pickled',
 'Boiled']

In [1192]:
rp_tokens = list_words_with_tag(tagged_recipe_names, "RP")
rp_tokens

['Hanout', 'Over']

In [1193]:
comma_tokens = list(set(list_words_with_tag(tagged_recipe_names, ",")))
comma_tokens

[',']

In [1194]:
for c in list(set(comma_tokens)):
  print(find_value_with_char(p_recipes, 'name', c))

['Bow Ties with Sausage, Tomatoes and Cream', 'Velveting Chicken Breast, Chinese Restaurant-Style', 'Chicken, Spinach, and Cheese Pasta Bake', 'Super-Simple, Super-Spicy Mongolian Beef', 'Creamy Potato, Carrot, and Leek Soup', 'Beef, Mushroom and Guinness Pie', 'Easy, Chewy Flourless Peanut Butter Cookies', 'Filipino Steamed Rice, Cebu Style', 'Orange, Honey and Soy Chicken', 'Chicken Francese, Italian-Style', 'Duck with Honey, Soy, and Ginger', 'Steak, Onion, and Pepper Fajitas', 'Indian Carrots, Peas and Potatoes', 'Simple, Baked Finnan Haddie', 'Indian-Style Rice with Cashews, Raisins and Turmeric', 'Serbian Ground Beef, Veggie, and Potato Bake', 'Fried Rice with Ginger, Hoisin, and Sesame', 'Chard Lentil Soup, Lebanese-Style', 'Easy, Cheesy Tortellini Bake', 'Curried Cashew, Pear, and Grape Salad', 'Pork, Sauerkraut and Dumplings', 'Spinach, Feta, and Pine Nut Ravioli Filling', 'Bell Pepper, Tomato, and Potato Indian Curry', 'Mascarpone Pasta with Chicken, Bacon and Spinach', 'Past

In [1195]:
jjr_tokens = list(set(list_words_with_tag(tagged_recipe_names, "JJR")))
jjr_tokens

['Healthier', 'Lighter']

In [1196]:
for j in list(set(jjr_tokens)):
  print(find_value_with_char(p_recipes, 'name', j))

['Healthier Bang Bang Chicken in the Air Fryer', 'Healthier Swedish Meatballs', 'Healthier Pan-Fried Honey-Sesame Chicken', 'Healthier Chicken Enchiladas I', 'Healthier Honey-Sesame Chicken']
['Lighter Mexican Meatloaf']


In [1197]:
jjs_tokens = list(set(list_words_with_tag(tagged_recipe_names, "JJS")))
jjs_tokens

['Oktoberfest']

In [1198]:
for j in list(set(jjs_tokens)):
  print(find_value_with_char(p_recipes, 'name', j))

['Oktoberfest Chicken and Red Cabbage', 'Oktoberfest Potato Salad', 'Oktoberfest Chili', 'The Recipes to Celebrate Oktoberfest']


In [1199]:
dt_tokens = list_words_with_tag(tagged_recipe_names, "DT")
dt_tokens

['a',
 'The',
 'No',
 'The',
 'the',
 'a',
 'the',
 'The',
 'the',
 'the',
 'a',
 'the',
 'the',
 'A',
 'a',
 'The',
 'a',
 'the',
 'the',
 'a',
 'a',
 'A',
 'The',
 'A',
 'the',
 'a',
 'a',
 'The',
 'a',
 'a',
 'The',
 'the',
 'The',
 'This',
 'The',
 'a',
 'a',
 'the',
 'The',
 'a',
 'a',
 'The',
 'a',
 'A',
 'the',
 'the',
 'No',
 'the',
 'a',
 'a',
 'The',
 'The',
 'a',
 'The',
 'the',
 'the',
 'The',
 'the',
 'a',
 'a',
 'The',
 'a',
 'the',
 'a',
 'The',
 'All',
 'The',
 'a',
 'the',
 'the',
 'the',
 'The',
 'The',
 'A',
 'a',
 'the',
 'a',
 'the',
 'The',
 'the',
 'a',
 'a',
 'a',
 'the',
 'a',
 'a',
 'the',
 'a',
 'An',
 'the',
 'a',
 'a',
 'a',
 'No',
 'a',
 'No']

In [1200]:
for dt in list(set(dt_tokens)):
  print(find_value_with_char(p_recipes, 'name', dt))

['Authentic German Potato Salad', 'Easy Authentic Mexican Rice', "Authentic Russian Salad 'Olivye'", 'Authentic Mexican Tortillas', 'The Original Donair From the East Coast of Canada', 'Authentic Paella Valenciana', 'Authentic Pad Thai', 'Lumpia in the Air Fryer', 'Refried Beans Without the Refry', 'Beef Stifado in the Slow Cooker', 'Authentic Mexican Breakfast Tacos', 'Authentic Enchiladas Verdes', 'Toad in the Hole', 'Authentic Chicken Tikka Masala', 'Authentic Mexican Enchiladas', 'Authentic Miso Soup', 'Healthier Bang Bang Chicken in the Air Fryer', 'Authentic Mexican Picadillo', 'Margaritas on the Rocks', 'Authentic French Meringues', 'Authentic Hungarian Goulash', 'Eggplant Parmesan For the Slow Cooker', 'Authentic Greek Moussaka', 'Authentic Chicken Madras', 'Authentic Thai Coconut Soup', 'Mongolian Beef from the Slow Cooker', 'Authentic and Easy Shrimp Curry', 'Authentic Patatas Bravas', 'Cuban Black Bean Soup in the Slow Cooker', 'Authentic Bangladeshi Beef Curry', 'Dave Matth

In [1201]:
to_tokens = list_words_with_tag(tagged_recipe_names, "TO")
to_tokens

['to', 'na', 'to', 'to', 'to', 'To', 'to', 'na', 'na', 'na']

In [1202]:
for to in list(set(to_tokens)):
  print(find_value_with_char(p_recipes, 'name', to))

['Super-Delicious Zuppa Toscana', 'Canadian Yellow Split Pea Soup with Ham', "Randy's Slow Cooker Ravioli Lasagna", 'Spinach Tomato Tortellini', 'Traditional Gyros', 'Cheese Lasagna', 'Creamy Chicken Lasagna', 'Chicken Parmigiana', 'French Canadian Tourtiere', 'Pipirrana', "Dash's Donair", 'American Lasagna', 'Taco Lasagna', "Bob's Stuffed Banana Peppers", 'Spaghetti alla Carbonara', 'Kalamata Olive Tapenade', 'Lyonnaise Potatoes', "Chef John's Lasagna", 'Original Homemade Italian Beef', 'Fettuccini Carbonara', 'Simply Traditional Lasagna', 'Spinach Cheese Manicotti', 'The Original Donair From the East Coast of Canada', 'Chicken and Shrimp Carbonara', 'Fried Empanadas', 'Authentic Paella Valenciana', 'Danish Cinnamon Snails', 'Panang Curry with Chicken', 'Easy Lasagna I', 'Eggplant Lasagna', 'Jamaican Fried Snapper', 'Mushrooms and Spinach Italian Style', 'Deep Dish Lasagna', 'Simple Spinach Lasagna', 'Quesadilla Salvadorena', 'Italian Chicken Marinade', 'Korean BBQ Chicken Marinade', 

Chicken is considered dollar?

In [1203]:
dol_tokens = list_words_with_tag(tagged_recipe_names, "$")
dol_tokens

['Chicken']

It's a tagging error, so this can be ignored

In [1204]:
for dol in dol_tokens:
  print(find_value_with_char(p_recipes, 'name', dol))

['Spicy Korean Fried Chicken with Gochujang Sauce', 'Greek Lemon Chicken and Potato Bake', "Chef John's Chicken Kiev", 'Indian-Style Chicken and Onions', 'Tender Italian Baked Chicken', 'Chicken Katsu', 'Chicken Stir-Fry', 'Mexican-Style Chicken Taco Casserole', 'Curry Stand Chicken Tikka Masala Sauce', 'Chicken Enchiladas V', 'Jamaican Style Curry Chicken', 'Salsa Chicken', 'Grilled Asian Chicken', 'Chicken Tikka Masala', 'Sweet and Sour Chicken I', 'Chicken Cordon Bleu II', 'Turkish Chicken Kebabs', 'Chicken Souvlaki with Tzatziki Sauce', 'Greek Lemon Chicken Soup', 'Chicken Cacciatore in a Slow Cooker', 'Chicken and Broccoli Stir-Fry', 'Creamy Chicken Lasagna', 'Broccoli and Chicken Stir-Fry', 'Chicken Parmigiana', 'Shoyu Chicken', 'Skillet Chicken Bulgogi', 'Easy Slow Cooker Chicken Tetrazzini', 'Sheet Pan Chicken Fajitas', 'White Chicken Enchilada Slow-Cooker Casserole', 'Chicken Enchiladas II', 'Chinese Chicken Fried Rice II', 'Chicken Milanese', 'Chicken Massaman Curry', "Chef J

There are some quotation marks

In [1205]:
quote_tokens = list_words_with_tag(tagged_recipe_names, "''")
quote_tokens

["''", "''", "'", "''", "''", "''", "''"]

Quotation marks are caused by possessive -'s

In [1206]:
for quote in quote_tokens:
  print(find_value_with_char(p_recipes, 'name', quote))

[]
[]
["Chef John's Chicken Kiev", "Angela's Awesome Enchiladas", "Randy's Slow Cooker Ravioli Lasagna", "'Chinese Buffet' Green Beans", "Chef John's Beef Rouladen", "Corned Beef and Cabbage Shepherd's Pie", "Gramma's Date Squares", "Authentic Russian Salad 'Olivye'", "Chef John's Meatless Meatballs", "Chef John's Beef Goulash", "Grandma's Noodles II", "Chef John's Clotted Cream", "Newfoundland Jigg's Dinner", "Chef John's Coq Au Vin", "Chef John's Loco Moco", "Dash's Donair", "Turkey Shepherd's Pie", "Papa Drexler's Bavarian Pretzels", "Bob's Stuffed Banana Peppers", "Chef John's Swedish Meatballs", "Chef John's German Recipes", "Chef John's Chicken Tikka Masala", "Maria's Mexican Rice", "Mom's Buttermilk Pancakes", "Geneva's Ultimate Hungarian Mushroom Soup", "Charley's Slow Cooker Mexican Style Meat", "Ingrid's Rouladen", "Chef John's Lasagna", "Lola's Horchata", "Chef John's Italian Sausage Chili", "Kid's Favorite Pizza Casserole", "Traci's Adobo Seasoning", "Frank's Favorite Slow-

 For now, leave the preprocessing of the recipe names first.

## Preprocessing of ingredients

In [1207]:
p_ingredients = []

for recipe in p_recipes:
    p_ingredients = p_ingredients + recipe['ingredients']
    
p_ingredients = list(set(p_ingredients))
len(p_ingredients)

19342

In [1208]:
p_ingredients[:10]

['2 tablespoons grated orange peel ',
 '1 green bell pepper, cut into matchstick-size pieces ',
 '2 cups dry bread crumbs ',
 '1 teaspoon New Mexico chili powder ',
 '1 teaspoon mayonnaise, or to taste ',
 '3\u2009½ tablespoons cornstarch ',
 '6 ounces dry miniature ravioli ',
 '2 cups peeled and cubed potatoes ',
 '1 bunch fresh basil, divided ',
 '10 sugar snap peas ']

In [1209]:
for i, ingre in enumerate(p_ingredients):
    p_ingredients[i] = p_ingredients[i].strip()

p_ingredients[:10]

['2 tablespoons grated orange peel',
 '1 green bell pepper, cut into matchstick-size pieces',
 '2 cups dry bread crumbs',
 '1 teaspoon New Mexico chili powder',
 '1 teaspoon mayonnaise, or to taste',
 '3\u2009½ tablespoons cornstarch',
 '6 ounces dry miniature ravioli',
 '2 cups peeled and cubed potatoes',
 '1 bunch fresh basil, divided',
 '10 sugar snap peas']

A reusable function that re-tags ingredients

In [1210]:
def retag_ingredients():
    tagged_recipe_ingredients = []

    for ingredient in p_ingredients:
        tagged_recipe_ingredients.append(tag_pos(ingredient))
        
    return tagged_recipe_ingredients

tagged_recipe_ingredients = retag_ingredients()
tagged_recipe_ingredients[:10]

[[('2', 'CD'),
  ('tablespoons', 'NNS'),
  ('grated', 'VBD'),
  ('orange', 'NN'),
  ('peel', 'NN')],
 [('1', 'CD'),
  ('green', 'JJ'),
  ('bell', 'NN'),
  ('pepper', 'NN'),
  (',', ','),
  ('cut', 'VBN'),
  ('into', 'IN'),
  ('matchstick-size', 'JJ'),
  ('pieces', 'NNS')],
 [('2', 'CD'),
  ('cups', 'NNS'),
  ('dry', 'JJ'),
  ('bread', 'NN'),
  ('crumbs', 'NNS')],
 [('1', 'CD'),
  ('teaspoon', 'JJ'),
  ('New', 'NNP'),
  ('Mexico', 'NNP'),
  ('chili', 'NN'),
  ('powder', 'NN')],
 [('1', 'CD'),
  ('teaspoon', 'NN'),
  ('mayonnaise', 'NN'),
  (',', ','),
  ('or', 'CC'),
  ('to', 'TO'),
  ('taste', 'VB')],
 [('3', 'CD'), ('½', 'JJ'), ('tablespoons', 'NNS'), ('cornstarch', 'VBP')],
 [('6', 'CD'),
  ('ounces', 'NNS'),
  ('dry', 'JJ'),
  ('miniature', 'NN'),
  ('ravioli', 'NN')],
 [('2', 'CD'),
  ('cups', 'NNS'),
  ('peeled', 'VBD'),
  ('and', 'CC'),
  ('cubed', 'VBD'),
  ('potatoes', 'NNS')],
 [('1', 'CD'),
  ('bunch', 'JJ'),
  ('fresh', 'JJ'),
  ('basil', 'NN'),
  (',', ','),
  ('divided', '

Numbers need a placeholder

In [1211]:
list_words_with_tag(tagged_recipe_ingredients, "CD")

['2',
 '1',
 '2',
 '1',
 '1',
 '3',
 '6',
 '2',
 '1',
 '10',
 '1',
 '1',
 '2',
 '1',
 '5',
 '5',
 '4',
 '1',
 '3',
 '3',
 '1',
 '4',
 '2',
 '1',
 '15',
 '2',
 '1',
 '15',
 '8',
 '1',
 '2',
 '3',
 '1',
 '1',
 '2',
 '1',
 '1',
 '4',
 '1',
 '1',
 '3',
 '1',
 '2',
 '4',
 '1',
 '1',
 '16',
 '9',
 'zapallo',
 '1',
 '6',
 '1',
 '20',
 '8',
 '4',
 '2',
 '1',
 '4',
 '2',
 '1',
 '2',
 '2',
 '4',
 '1',
 '1',
 '5',
 '6',
 '1',
 '5',
 '1',
 '2',
 '1',
 '1',
 '16',
 '1',
 '6',
 '5',
 '8',
 '2',
 '2',
 '14.5',
 '2',
 '3',
 '1',
 '2',
 '3',
 '6',
 '4',
 '1',
 '1',
 '8',
 '1',
 '1',
 '2',
 '1',
 '1',
 '3',
 '2',
 '1',
 '2',
 '2',
 '1',
 '15',
 '1',
 '1',
 '1',
 '1',
 '4',
 '2',
 '1',
 '10',
 '2',
 '16',
 '6',
 '11',
 '6',
 '1',
 '1',
 '1',
 '1',
 '20',
 '3',
 '24',
 '1',
 '1',
 '2',
 '1/2',
 '1',
 '2',
 '1',
 '1',
 '2',
 '1',
 '2',
 '1',
 '14',
 '2',
 '14.5',
 '1',
 '16',
 '1',
 '8',
 '4',
 '1',
 '4',
 '8',
 '1',
 '12',
 '1',
 '1',
 '12',
 '1/2',
 '1',
 '1',
 '36',
 '1',
 '3',
 '1',
 '4',
 '1',
 '2',
 

NLTK assumes fractions as JJ (adjectives)

In [1212]:
list_words_with_tag(tagged_recipe_ingredients, "JJ")

['green',
 'matchstick-size',
 'dry',
 'teaspoon',
 '½',
 'dry',
 'bunch',
 'fresh',
 '½',
 '¼',
 '½',
 'white',
 '⅔',
 'frozen',
 'brown',
 '½',
 'green',
 '½',
 'vegetable',
 'garlic',
 'whole',
 'olive',
 'garlic',
 'such',
 'no-salt-added',
 'black',
 'pita',
 '¼',
 '1-inch',
 'pitted',
 '½',
 'green',
 '¼',
 '½',
 'sweet',
 'such',
 '¼',
 'jalapeno',
 'fresh',
 '½',
 'white',
 'dark',
 'brown',
 'large',
 'fresh',
 'hot',
 '½',
 'unsalted',
 '½',
 'vegetable',
 'vegetable',
 'red',
 'green',
 '1-inch',
 'white',
 '¼',
 'garnish',
 'hot',
 'salt',
 '½',
 'chopped',
 'cumin',
 'teaspoon',
 'extra-virgin',
 'small',
 'red',
 '½',
 'inch',
 'fresh',
 '¼',
 'pimento-stuffed',
 'green',
 'whole',
 '⅓',
 'fresh',
 'chili-seasoned',
 'hot',
 '¾',
 'garlic',
 'ancho',
 'chipotle',
 'fresh',
 'small',
 '½',
 'Mexican',
 'Korean',
 'red',
 'frozen',
 'mixed',
 'stir-fry',
 '¾',
 'hot',
 'short',
 '⅓',
 'Japanese',
 'sweet',
 'rice',
 'red',
 '½',
 'loaf',
 'hard',
 'Italian',
 '⅛',
 'olive',

Create a function that converts any fraction in a text to integer

In [1213]:
import unicodedata
from decimal import Decimal

def fraction_to_int(text):
  for i, char in enumerate(text):
    try:
      # unicode.numeric converts fractions such as ½ to decimal place, 0.25
      # remove trailing decimals, otherwise keep decimals
      text = text[:i] + str(Decimal(unicodedata.numeric(char)).normalize()) + text[i + 1:]
    except Exception as e:
      pass
  # Because number + fraction, such as 1 1/4 may be converted to 1 0, so use re.sub to remove
  text = re.sub("([0-9]+ [0])+", "4", text)
  return text

for i, ingre in enumerate(p_ingredients):
    p_ingredients[i] = fraction_to_int(p_ingredients[i])

tagged_recipe_ingredients = retag_ingredients()
p_ingredients[:20]

['2 tablespoons grated orange peel',
 '1 green bell pepper, cut into matchstick-size pieces',
 '2 cups dry bread crumbs',
 '1 teaspoon New Mexico chili powder',
 '1 teaspoon mayonnaise, or to taste',
 '4.5 tablespoons cornstarch',
 '6 ounces dry miniature ravioli',
 '2 cups peeled and cubed potatoes',
 '1 bunch fresh basil, divided',
 '10 sugar snap peas',
 '0.5 tablespoon hoisin sauce',
 '1 pinch ground ginger',
 'Salt to taste',
 '1 cup sesame seeds',
 '2 slices bread',
 '0.25 cup drained canned peas',
 '0.5 teaspoon white sugar, or to taste',
 '0.6666666666666666296592325125 cup milk cracker crumbs',
 '1 (5 ounce) can tuna',
 '5 teaspoons kosher salt']

By converting fractions into integers, NLTK stops seeing them as adjectives (JJ) and instead, they are considered numbers (CD)

In [1214]:
list_words_with_tag(tagged_recipe_ingredients, "JJ")

['green',
 'matchstick-size',
 'dry',
 'teaspoon',
 'dry',
 'bunch',
 'fresh',
 'white',
 'frozen',
 'brown',
 'green',
 'vegetable',
 'garlic',
 'whole',
 'olive',
 'garlic',
 'such',
 'no-salt-added',
 'black',
 'pita',
 '1-inch',
 'pitted',
 'green',
 'skinless',
 'boneless',
 'sweet',
 'such',
 'jalapeno',
 'fresh',
 'white',
 'dark',
 'brown',
 'large',
 'fresh',
 'hot',
 'unsalted',
 'vegetable',
 'vegetable',
 'red',
 'green',
 '1-inch',
 'white',
 'garnish',
 'hot',
 'salt',
 'chopped',
 'cumin',
 'teaspoon',
 'extra-virgin',
 'small',
 'red',
 'inch',
 'fresh',
 'pimento-stuffed',
 'green',
 'whole',
 'fresh',
 'chili-seasoned',
 'hot',
 'cup',
 'garlic',
 'ancho',
 'chipotle',
 'fresh',
 'small',
 'Mexican',
 'Korean',
 'red',
 'frozen',
 'mixed',
 'stir-fry',
 'hot',
 'short',
 'Japanese',
 'sweet',
 'rice',
 'red',
 'loaf',
 'day-old',
 'hard',
 'Italian',
 'cup',
 'olive',
 'small',
 'purple',
 'olive',
 'soft',
 'yellow',
 'fresh',
 'fresh',
 'tablespoon',
 'whole',
 'bla

Replace all the numbers with placeholder of 4

In [1215]:
for i, ingre in enumerate(p_ingredients):
    p_ingredients[i] = searchReplacePatt(p_ingredients[i], NUMPATTERN, "4")
    
tagged_recipe_ingredients = retag_ingredients()
p_ingredients[:20]

['4 tablespoons grated orange peel',
 '4 green bell pepper, cut into matchstick-size pieces',
 '4 cups dry bread crumbs',
 '4 teaspoon New Mexico chili powder',
 '4 teaspoon mayonnaise, or to taste',
 '4.4 tablespoons cornstarch',
 '4 ounces dry miniature ravioli',
 '4 cups peeled and cubed potatoes',
 '4 bunch fresh basil, divided',
 '4 sugar snap peas',
 '4.4 tablespoon hoisin sauce',
 '4 pinch ground ginger',
 'Salt to taste',
 '4 cup sesame seeds',
 '4 slices bread',
 '4.4 cup drained canned peas',
 '4.4 teaspoon white sugar, or to taste',
 '4.4 cup milk cracker crumbs',
 '4 (4 ounce) can tuna',
 '4 teaspoons kosher salt']

In [1216]:
new_cd_tokens = list(set(list_words_with_tag(tagged_recipe_ingredients, "CD")))
new_cd_tokens.remove('4')
new_cd_tokens

['beef4',
 'mostaccioli',
 'bleu',
 'millet',
 'provolone',
 '4.4',
 'one',
 'yellow',
 '4.4.4',
 'fontina',
 'ziti',
 '4up',
 "za'atar",
 'kalamata',
 'marinara',
 'seven',
 'mascarpone',
 'mozzarella',
 'zucchini',
 'four',
 'yum',
 '4/4x4/4',
 '4/4',
 '4p',
 'xanthan',
 'zapallo']

In [1217]:
def find_ingre_with_substring(sub):
    ingres = []
    for ingre in p_ingredients:
        matches = searchWordsPatt(ingre, sub)
        if len(matches)  > 0:
            ingres.append(ingre)
    return ingres

find_ingre_with_substring('4/4')

['4 (4 pound) boneless pork loin, cut into 4/4 inch strips',
 '4 (4 ounce) pork tenderloin, cut into 4/4 inch cubes',
 '4 fresh green beans, cut into 4/4 inch pieces',
 '4.4 cups warm wate4 degrees4/4 degrees C)',
 '4 medium eggplants, cut in half lengthwise and hollowed out to 4/4 inch flesh rim',
 '4 (4 inch) corn tortillas, cut into 4/4 inch strips',
 '4 red bell pepper, cut into 4/4 inch pieces',
 '4 (4 ounce) packages firm tofu, drained and cut into 4/4-inch slices',
 '4 leeks, white part only, cut into 4/4 inch pieces',
 '4 pound hot Italian turkey sausage, cut into 4/4 inch slices',
 '4 pounds boneless beef chuck, cut into 4-4/4-inch pieces',
 '4 (4 ounce) can whole tomatoes, drained, 4/4 cup juice reserved',
 '4 pounds new potatoes, cut into 4/4 inch thick pieces',
 '4 leek, split in half, then cut into 4/4-inch pieces',
 '4 eggplant - trimmed, peeled, and sliced 4/4-inch thick',
 '4 pounds boneless lamb shoulder, cut into 4/4-inch-thick strips',
 '4.4 red bell pepper, sliced i

In [1218]:
search_edit_ingredient(r"®", "")

find_ingre_with_substring('®')

[]

In [1219]:
def search_edit_ingredient(regex, new_val):
    for i, ingre in enumerate(p_ingredients):
        p_ingredients[i] = searchReplacePatt(p_ingredients[i], regex, new_val)
        
search_edit_ingredient(r"4/4", "4.4")

find_ingre_with_substring('4/4')

[]

In [1220]:
find_ingre_with_substring('4p')

['4.4 c4p4.4-inch long vermicelli',
 '4.4 4p warm milk (4 degrees F/4 degrees C)']

In [1221]:
search_edit_ingredient(r"c4p", "")
search_edit_ingredient(r"4p", "")

find_ingre_with_substring('4p')

[]

In [1222]:
find_ingre_with_substring('4up')

['4.4 4up 4% milk']

In [1223]:
search_edit_ingredient(r"4up", "7up")

find_ingre_with_substring('7up')

['4.4 7up 4% milk']

In [1224]:
def split_ingre_to_two(target, search, retain_target=False):
    for i, ingre in enumerate(p_ingredients):
        if p_ingredients[i] == target:
            splits = re.split(search, p_ingredients[i])
            new_ingre1 = splits[0].strip()
            new_ingre2 = splits[1].strip()
            if retain_target:
                new_ingre2 = search.strip()
            del p_ingredients[i]
            p_ingredients.append(new_ingre1)
            p_ingredients.append(new_ingre2)

split_ingre_to_two('4.4 7up 4% milk', " 4% milk", retain_target=True)

find_ingre_with_substring('7up')

['4.4 7up']

In [1225]:
tagged_recipe_ingredients = retag_ingredients()
p_ingredients[:20]

['4 tablespoons grated orange peel',
 '4 green bell pepper, cut into matchstick-size pieces',
 '4 cups dry bread crumbs',
 '4 teaspoon New Mexico chili powder',
 '4 teaspoon mayonnaise, or to taste',
 '4.4 tablespoons cornstarch',
 '4 ounces dry miniature ravioli',
 '4 cups peeled and cubed potatoes',
 '4 bunch fresh basil, divided',
 '4 sugar snap peas',
 '4.4 tablespoon hoisin sauce',
 '4 pinch ground ginger',
 'Salt to taste',
 '4 cup sesame seeds',
 '4 slices bread',
 '4.4 cup drained canned peas',
 '4.4 teaspoon white sugar, or to taste',
 '4.4 cup milk cracker crumbs',
 '4 (4 ounce) can tuna',
 '4 teaspoons kosher salt']

In [1226]:
new_cd_tokens = list(set(list_words_with_tag(tagged_recipe_ingredients, "CD")))
new_cd_tokens

['beef4',
 'mostaccioli',
 'bleu',
 'millet',
 'provolone',
 '4',
 '4.4',
 'one',
 'yellow',
 '4.4.4',
 'fontina',
 'ziti',
 "za'atar",
 'kalamata',
 'marinara',
 'seven',
 'mascarpone',
 'mozzarella',
 '4.4x4.4',
 'zucchini',
 'four',
 'yum',
 '7up',
 'xanthan',
 'zapallo']

In [1227]:
tagged_recipe_ingredients = retag_ingredients()

all_ingre_tags = []

for POS in ALL_POS:
  new_dic = {POS: list_words_with_tag(tagged_recipe_ingredients, POS)}
  all_ingre_tags.append(new_dic)

get_tag_number(all_ingre_tags)

[{'$': 0},
 {"''": 14},
 {'(': 3744},
 {')': 3828},
 {',': 8512},
 {'--': 0},
 {'.': 23},
 {':': 304},
 {'CC': 3074},
 {'CD': 21788},
 {'DT': 99},
 {'EX': 0},
 {'FW': 52},
 {'IN': 2849},
 {'JJ': 13401},
 {'JJR': 523},
 {'JJS': 6},
 {'LS': 0},
 {'MD': 612},
 {'NN': 32987},
 {'NNP': 2411},
 {'NNPS': 2},
 {'NNS': 13598},
 {'PDT': 1},
 {'POS': 126},
 {'PRP': 2},
 {'PRP$': 1},
 {'RB': 1452},
 {'RBR': 5},
 {'RBS': 0},
 {'RP': 13},
 {'SYM': 53},
 {'TO': 1039},
 {'UH': 0},
 {'VB': 1725},
 {'VBD': 8949},
 {'VBG': 354},
 {'VBN': 3434},
 {'VBP': 646},
 {'VBZ': 588},
 {'WDT': 1},
 {'WP': 0},
 {'WP$': 0},
 {'WRB': 0},
 {'``': 0}]

In [1228]:
colon_tags = list(set(list_words_with_tag(tagged_recipe_ingredients, ":")))
colon_tags

[':', '--', ';', '-']

In [1229]:
for c in colon_tags:
    print(find_ingre_with_substring(c))

['Spice Blend:', 'Chipotle Mayonnaise:', 'Meatballs:', 'Caramel:', 'Fillings:', 'Gravy:', 'Dipping Sauce:']
['4 large skinless, boneless chicken breast halves -- trimmed and cut into 4-inch pieces']
['4 cups assorted mushrooms, sliced (I like white buttons, oyster, shiitake, portobello and crimini; if using shiitake, discard stems)', '4 raw chop with refuse, 4 g; (blank) 4.4 ounces boneless pork chops, pounded to 4.4 inch thick', '4 (4 ounce) can black beans; drain and reserve liquid']
['4 green bell pepper, cut into matchstick-size pieces', '4 (4 ounce) can no-salt-added black beans, drained and rinsed', '4 skinless, boneless chicken thighs, cut into 4-inch pieces', '4 bunches green onions, cut into 4-inch pieces', '4 teaspoon extra-virgin coconut oil', '4.4 cup chopped pimento-stuffed green Manzanilla olives', '4 (4.4 ounce) cans chili-seasoned diced tomatoes', '4 mango - peeled, seeded, and chopped', '4 ancho chile pepper - stems, seeds, and veins removed', '4 cups frozen mixed stir

In [1230]:
find_ingre_with_substring("--")

['4 large skinless, boneless chicken breast halves -- trimmed and cut into 4-inch pieces']

In [1231]:
search_edit_ingredient(r"--", ",")

find_ingre_with_substring('--')

[]

In [1232]:
find_ingre_with_substring(":")

['Spice Blend:',
 'Chipotle Mayonnaise:',
 'Meatballs:',
 'Caramel:',
 'Fillings:',
 'Gravy:',
 'Dipping Sauce:']

In [1233]:
search_edit_ingredient(r":", "")

find_ingre_with_substring(':')

[]

In [1234]:
find_ingre_with_substring(";")

['4 cups assorted mushrooms, sliced (I like white buttons, oyster, shiitake, portobello and crimini; if using shiitake, discard stems)',
 '4 raw chop with refuse, 4 g; (blank) 4.4 ounces boneless pork chops, pounded to 4.4 inch thick',
 '4 (4 ounce) can black beans; drain and reserve liquid']

In [1235]:
find_ingre_with_substring(', 4 g')

['4 raw chop with refuse, 4 g; (blank) 4.4 ounces boneless pork chops, pounded to 4.4 inch thick']

In [1236]:
search_edit_ingredient(r", 4 g; \(blank\)", ", 4g")

find_ingre_with_substring(";")

['4 cups assorted mushrooms, sliced (I like white buttons, oyster, shiitake, portobello and crimini; if using shiitake, discard stems)',
 '4 (4 ounce) can black beans; drain and reserve liquid']

In [1237]:
split_ingre_to_two('4 raw chop with refuse, 4g; (blank) 4.4 ounces boneless pork chops, pounded to 4.4 inch thick', "; ")

find_ingre_with_substring(";")

['4 cups assorted mushrooms, sliced (I like white buttons, oyster, shiitake, portobello and crimini; if using shiitake, discard stems)',
 '4 (4 ounce) can black beans; drain and reserve liquid']

In [1238]:
split_ingre_to_two("4 cups assorted mushrooms, sliced (I like white buttons, oyster, shiitake, portobello and crimini; if using shiitake, discard stems)", r"\(I like ")

find_ingre_with_substring(";")

['4 (4 ounce) can black beans; drain and reserve liquid',
 'white buttons, oyster, shiitake, portobello and crimini; if using shiitake, discard stems)']

In [1239]:
find_ingre_with_substring("/")

['4.4 c4 warm water (4 degrees F/4 degrees C)',
 '4 cups warm water (4 degrees F/4 degrees C)',
 '4 tablespoons warm water (4 degrees F/4 degrees C)',
 '4 cups warm water (4 degrees F/4 degrees C)',
 '4.4 c4 warm water (4 degrees F/4 degrees C)',
 '4.4 tablespoon Guacamole, salsa, and/or sour cream',
 '4 cup warm milk (4 degrees F/4 degrees C)',
 '4.4 cu4 warm water (4 degrees F/4 degrees C)',
 '4 cup warm water (4 degrees F/4 degrees C)',
 '4 cup warm water (4 degrees F/4 degrees C)',
 '4.4 cups warm wat4(4 degree4F/4 degrees C)',
 '4 cups warm water (4 degrees F/4 degrees C)',
 '4 (4 ounce) package round gyoza/potsticker wrappers',
 '4.4  warm milk (4 degrees F/4 degrees C)',
 '4 tablespoons warm milk (4 degrees F/4 degrees C)',
 '4 cup shredded Cheddar/Monterey Jack cheese blend',
 '4 (4.4 ounce) package corn bread/muffin mix',
 '4 tablespoons warm water (4 degrees F/4 degrees C)']

In [1240]:
search_edit_ingredient(r"\/", " or ")
find_ingre_with_substring("/")

[]

In [1241]:
tagged_recipe_ingredients = retag_ingredients()

tagged_recipe_ingredients[:20]

[[('4', 'CD'),
  ('tablespoons', 'NNS'),
  ('grated', 'VBD'),
  ('orange', 'NN'),
  ('peel', 'NN')],
 [('4', 'CD'),
  ('green', 'JJ'),
  ('bell', 'NN'),
  ('pepper', 'NN'),
  (',', ','),
  ('cut', 'VBN'),
  ('into', 'IN'),
  ('matchstick-size', 'JJ'),
  ('pieces', 'NNS')],
 [('4', 'CD'),
  ('cups', 'NNS'),
  ('dry', 'JJ'),
  ('bread', 'NN'),
  ('crumbs', 'NNS')],
 [('4', 'CD'),
  ('teaspoon', 'JJ'),
  ('New', 'NNP'),
  ('Mexico', 'NNP'),
  ('chili', 'NN'),
  ('powder', 'NN')],
 [('4', 'CD'),
  ('teaspoon', 'NN'),
  ('mayonnaise', 'NN'),
  (',', ','),
  ('or', 'CC'),
  ('to', 'TO'),
  ('taste', 'VB')],
 [('4.4', 'CD'), ('tablespoons', 'NNS'), ('cornstarch', 'NN')],
 [('4', 'CD'),
  ('ounces', 'NNS'),
  ('dry', 'JJ'),
  ('miniature', 'NN'),
  ('ravioli', 'NN')],
 [('4', 'CD'),
  ('cups', 'NNS'),
  ('peeled', 'VBD'),
  ('and', 'CC'),
  ('cubed', 'VBD'),
  ('potatoes', 'NNS')],
 [('4', 'CD'),
  ('bunch', 'JJ'),
  ('fresh', 'JJ'),
  ('basil', 'NN'),
  (',', ','),
  ('divided', 'VBD')],
 [('

## Examining other POS in ingredients

In [1242]:
fw_tags = list(set(list_words_with_tag(tagged_recipe_ingredients, "FW")))
fw_tags

['di',
 'vanilla',
 'mirin',
 'herbes',
 'kalonji',
 'arbol',
 'miso',
 'pico',
 'bilbao',
 'skin',
 'kalamansi',
 's',
 'kielbasa',
 'paprika',
 'de',
 'gallo']

In [1243]:
rp_tags = list(set(list_words_with_tag(tagged_recipe_ingredients, "RP")))
rp_tags

['tomato', 'out', 'aside', 'off', 'up', 'dashi']

In [1244]:
for rp in rp_tags:
    print(find_ingre_with_substring(" " + rp))

['4 roma (plum) tomatoes, chopped', '4 Roma (plum) tomatoes, diced', '4 Roma tomatoes', '4 (4.4 ounce) cans chili-seasoned diced tomatoes', '4 tomatoes, chopped', '4 pound tomatoes, peeled and chopped', '4 (4.4 ounce) cans diced tomatoes, drained', '4 cups chopped tomatoes', '4 Roma tomatoes, sliced', '4 pound Roma tomatoes', '4 cup tomato puree', '4 (4 ounce) can diced tomatoes', '4 (4.4 ounce) can diced tomatoes with juice', '4 cups tomato juice', '4 (4 ounce) jar tomato pasta sauce', '4 (4 ounce) can peeled, chopped tomatoes', '4.4 cups canned tomato sauce', '4 large tomatoes, peeled, seeded, and sliced into strips', '4 roma (plum) tomatoes, chopped', '4 (4.4 ounce) can peeled and diced tomatoes with juice', '4 (4 ounce) can whole Italian plum tomatoes (preferably San Marzano)', '4 (4 ounce) can Italian-style plum tomatoes, drained and coarsely chopped', '4 teaspoons minced tomato', '4 cups chopped tomatoes, pushed through a sieve', '4 medium ripe tomatoes, cut into wedges', '4 toma

In [1245]:
rbr_tags = list(set(list_words_with_tag(tagged_recipe_ingredients, "RBR")))
rbr_tags

['leeks', 'lamb', 'lobster']

In [1246]:
wdt_tags = list(set(list_words_with_tag(tagged_recipe_ingredients, "WDT")))
wdt_tags

['whole']

In [1247]:
pdt_tags = list(set(list_words_with_tag(tagged_recipe_ingredients, "PDT")))
pdt_tags

['half']

In [1248]:
prp_tags = list(set(list_words_with_tag(tagged_recipe_ingredients, "PRP")))
prp_tags

['you']

In [1249]:
find_ingre_with_substring("you ")

['4 (4 ounce) packages garlic and herb couscous mix (or any flavor you prefer)']

In [1250]:
prp_tags = list(set(list_words_with_tag(tagged_recipe_ingredients, "PRP$")))
prp_tags

['your']

In [1251]:
find_ingre_with_substring("your")

['4 (4 ounce) package pasta, your choice of shape']

In [1252]:
punc_tags = list(set(list_words_with_tag(tagged_recipe_ingredients, ".")))
punc_tags

['!', '.']

In [1253]:
find_ingre_with_substring("!")

['4.4 cup Greek salad dressing, such as Yazzo!']

In [1254]:
quote_tags = list(set(list_words_with_tag(tagged_recipe_ingredients, "''")))
quote_tags

["''"]

In [1255]:
for q in quote_tags:
    print(find_ingre_with_substring("'"))

["4 teaspoons garlic pepper seasoning (such as SuzyQ's Santa Maria Valley Style Seasoning), or to taste", "4 cups parboiled rice (such as Uncle Ben's)", "4 cup unsweetened cocoa powder (such as Hershey's)", "4 cups confectioners' sugar", "4 cups passion fruit juice (such as Welch's)", "4.4 cup confectioners' sugar for rolling", "4.4 cup confectioners' sugar for decoration", "4.4 cup confectioners' sugar, or as needed", "4.4 cups pearl sugar (such as Lars' Own)", "4.4 teaspoon seasoned salt (such as Johnny's Seasoning Salt)", "4 tablespoons za'atar", "4.4 cup Parmesan and Romano cheese blend (such as Sargento's), divided", "4 (4 ounce) package sweet baking chocolate (such as Baker's German's Sweet Chocolate) - broken into pieces", "4.4 cups durum wheat semolina flour (such as Bob's Red Mill)", "4.4 tablespoon pepper sauce (such as Frank's Red Hot)", "4.4 cups confectioners' sugar, sifted, or more as needed", "4 (4.4 ounce) package UNCLE BEN'S Jasmine READY RICE", "4.4 cups confectioners

In [1256]:
list(set(list_words_with_tag(tagged_recipe_ingredients, "SYM")))

['mangoes',
 'lemon',
 'lettuce',
 'avocado',
 'basil',
 'choy',
 'tomato',
 'thighs',
 'spinach',
 'mangos',
 'cucumber',
 'beaten',
 'sauerkraut',
 'kale',
 'avocados',
 'squash',
 'shrimp',
 'leeks',
 'mango',
 'breast']

## Casing of recipe names

In [1257]:
all_recipe_names = []

for recipe in p_recipes:
    try:
        all_recipe_names.append(recipe['name'])
    except Exception as e:
        pass
    
all_recipe_names[:10]

['Pan-Fried Asparagus',
 'Creamy Au Gratin Potatoes',
 'Super-Delicious Zuppa Toscana',
 'Simple Teriyaki Sauce',
 'Spicy Korean Fried Chicken with Gochujang Sauce',
 'Spaghetti Aglio e Olio',
 'Easy Garam Masala',
 'Easy Chorizo Street Tacos',
 'Russian Cabbage Rolls with Gravy',
 'Shrimp Scampi with Pasta']

In [1258]:
all_recipe_names_corpus = ("\n").join(all_recipe_names)

all_recipe_names_corpus

'Pan-Fried Asparagus\nCreamy Au Gratin Potatoes\nSuper-Delicious Zuppa Toscana\nSimple Teriyaki Sauce\nSpicy Korean Fried Chicken with Gochujang Sauce\nSpaghetti Aglio e Olio\nEasy Garam Masala\nEasy Chorizo Street Tacos\nRussian Cabbage Rolls with Gravy\nShrimp Scampi with Pasta\nGreek Lemon Chicken and Potato Bake\nEasy Mexican Casserole\nGerman Apple Cake I\nSpanish Flan\nGerman Pork Chops and Sauerkraut\nSpaghetti Cacio e Pepe\nChef John\'s Chicken Kiev\nIndian-Style Chicken and Onions\nFajita Seasoning\nPerfect Sushi Rice\nTender Italian Baked Chicken\nAuthentic German Potato Salad\nMiso Soup\nMexican Rice II\nSpongy Japanese Cheesecake\nChicken Katsu\nChicken Stir-Fry\nQuick Beef Stir-Fry\nEasy Authentic Mexican Rice\nHerbs de Provence\nGreek or House Dressing\nFrench Bread\nFocaccia Bread\nJamaican Fried Dumplings\nGluehwein\nCoquilles Saint-Jacques\nMexican-Style Chicken Taco Casserole\nRosemary Braised Lamb Shanks\nMake-Ahead Vegetarian Moroccan Stew\nCurry Stand Chicken Tikka

In [1259]:
import nltk

recipe_tokens = list(set(nltk.word_tokenize(all_recipe_names_corpus)))
recipe_tokens[:10]

['Margaritas',
 'Devonshire',
 'Prune',
 'Sour',
 'Sukha',
 'Daal',
 'Caribbean-Spiced',
 'Haggis',
 'Orange',
 'Kalamatas']

In [1260]:
len(recipe_tokens)

3271

In [1261]:
ingredients_corpus = ("\n").join(p_ingredients)

ingredients_corpus

'4 tablespoons grated orange peel\n4 green bell pepper, cut into matchstick-size pieces\n4 cups dry bread crumbs\n4 teaspoon New Mexico chili powder\n4 teaspoon mayonnaise, or to taste\n4.4 tablespoons cornstarch\n4 ounces dry miniature ravioli\n4 cups peeled and cubed potatoes\n4 bunch fresh basil, divided\n4 sugar snap peas\n4.4 tablespoon hoisin sauce\n4 pinch ground ginger\nSalt to taste\n4 cup sesame seeds\n4 slices bread\n4.4 cup drained canned peas\n4.4 teaspoon white sugar, or to taste\n4.4 cup milk cracker crumbs\n4 (4 ounce) can tuna\n4 teaspoons kosher salt\n4 cups frozen hash brown potatoes, thawed\n4 ounce lychee juice (from a can of lychees)\n4.4 cup chopped orange bell pepper\n4 green onions, thinly sliced\n4.4 quart vegetable oil for frying\n4 cloves garlic, whole\n4 head lettuce, shredded\n4 tablespoons olive oil, divided\n4 teaspoons garlic pepper seasoning (such as SuzyQ\'s Santa Maria Valley Style Seasoning), or to taste\n4 (4 ounce) can no-salt-added black beans, d

In [1262]:
ingre_tokens = list(set(nltk.word_tokenize(ingredients_corpus)))
ingre_tokens[:10]

['preserves',
 'chunks',
 'bran',
 'flakes',
 'huckleberries',
 'asafoetida',
 'Sour',
 'quinces',
 'farmer',
 'Parmigiano']

In [1263]:
len(ingre_tokens)

2815

Most words in recipe tokens are capitalized

In [1264]:
lower_recipe_tokens = []
for token in recipe_tokens:
    if token[0].islower():
        lower_recipe_tokens.append(token)
        
lower_recipe_tokens

['on',
 'su',
 "all'Amatriciana",
 'and',
 'y',
 'chili',
 'la',
 'version',
 'to',
 'au',
 'en',
 'alla',
 'de',
 'from',
 'a',
 'the',
 'na',
 'or',
 'of',
 'et',
 'nach',
 'bil',
 'aux',
 'powder',
 'z',
 'by',
 'di',
 'al',
 'e',
 'sa',
 'its',
 'el',
 'con',
 'over',
 'with',
 'des',
 "l'Oignon",
 'in',
 'laziale',
 'for',
 'without',
 'aka',
 'le']

Number of words that are not capitalized increased significantly crosschecking with lowercase words in ingredient tokens

In [1265]:
for i, name in enumerate(recipe_tokens):
    for ingre in ingre_tokens:
        if recipe_tokens[i].lower() == ingre:
            recipe_tokens[i] = recipe_tokens[i].lower()

lower_recipe_tokens = []
for token in recipe_tokens:
    if token[0].islower():
        lower_recipe_tokens.append(token)
        
len(lower_recipe_tokens)

923

In [1266]:
upper_recipe_tokens = list(filter(str.istitle, recipe_tokens))
len(upper_recipe_tokens)

2314

In [1267]:
upper_recipe_tokens[:20]

['Margaritas',
 'Devonshire',
 'Sukha',
 'Daal',
 'Caribbean-Spiced',
 'Haggis',
 'Kalamatas',
 'Christmas',
 'Kofta',
 'Muffins',
 'Wow',
 'Veggie',
 'Giniling',
 'Jalisco',
 'Blueberry',
 'Indian',
 'Chee',
 'Fereni',
 'Pebre',
 'Kouign-Amann']

In [1268]:
!pip install country_list

install: missing destination file operand after 'country_list'
Try 'install --help' for more information.


In [1269]:
countries = dict(countries_for_language('en'))
countries = list(countries.values())

countries

['Afghanistan',
 'Åland Islands',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua & Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia & Herzegovina',
 'Botswana',
 'Bouvet Island',
 'Brazil',
 'British Indian Ocean Territory',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Caribbean Netherlands',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Christmas Island',
 'Cocos (Keeling) Islands',
 'Colombia',
 'Comoros',
 'Congo - Brazzaville',
 'Congo - Kinshasa',
 'Cook Islands',
 'Costa Rica',
 'Côte d’Ivoire',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egyp

In [1270]:
countries = ' '.join([elem for elem in countries])
countries = countries.replace('&', '')
countries = countries.split(" ")
countries = [i.strip() for i in countries]
countries = [string for string in countries if string != ""]
countries = [string for string in countries if string != "-"]

countries = countries + ["Filipino", "Malay", "Spanish", "Danish", "Welsh"]

countries

['Afghanistan',
 'Åland',
 'Islands',
 'Albania',
 'Algeria',
 'American',
 'Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua',
 'Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia',
 'Herzegovina',
 'Botswana',
 'Bouvet',
 'Island',
 'Brazil',
 'British',
 'Indian',
 'Ocean',
 'Territory',
 'British',
 'Virgin',
 'Islands',
 'Brunei',
 'Bulgaria',
 'Burkina',
 'Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape',
 'Verde',
 'Caribbean',
 'Netherlands',
 'Cayman',
 'Islands',
 'Central',
 'African',
 'Republic',
 'Chad',
 'Chile',
 'China',
 'Christmas',
 'Island',
 'Cocos',
 '(Keeling)',
 'Islands',
 'Colombia',
 'Comoros',
 'Congo',
 'Brazzaville',
 'Congo',
 'Kinshasa',
 'Cook',
 'Islands',
 'Costa',
 'Rica',
 'Côte',
 'd’Ivoire',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'C

In [1271]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

porter = PorterStemmer()
lancaster=LancasterStemmer()

porter_c = []
lancester_c = []

for c in countries:
    port = porter.stem(c.split(' ').pop(0))
    if len(port) < 5:
        port = c[:4]
    porter_c.append(port.capitalize())
    lan = lancaster.stem(c.split(' ').pop(0))
    if len(lan) < 5:
        lan = c[:4]
    lancester_c.append(lan.capitalize())
    
print(porter_c[:10])
print(lancester_c[:10])

['Afghanistan', 'Åland', 'Island', 'Albania', 'Algeria', 'American', 'Samoa', 'Andorra', 'Angola', 'Anguilla']
['Afgh', 'Åland', 'Island', 'Alban', 'Alger', 'Amer', 'Samo', 'Andorr', 'Angol', 'Anguill']


In [1272]:
token_with_country_prefix = []
for rt in recipe_tokens:
    for lan in lancester_c:
        if lan in rt:
            token_with_country_prefix.append(rt)

token_with_country_prefix = sorted(list(set(token_with_country_prefix)))
token_with_country_prefix.remove("No-Cook")
token_with_country_prefix

["'Chinese",
 'Afghan',
 'Afghani',
 'African',
 'African-Style',
 'Afritada',
 'Algerian',
 'Almond-Ricotta',
 'American',
 'Americano',
 'Arabic',
 'Argentine',
 'Argentinean',
 'Armenian',
 'Australian',
 'Bangladeshi',
 'Belgi',
 'Belgian',
 'Belizean',
 'Bermuda',
 'Bhutanese',
 'Bolivian',
 'Brazilian',
 'Brazilian-Style',
 'British',
 'Bulgarian',
 'Cambodian',
 'Canada',
 'Canadian',
 'Cape',
 'Capezzoli',
 'Caribbean',
 'Caribbean-Spiced',
 'Chad',
 'Chilaquiles',
 'Chilean',
 'Chilean-Style',
 'Chinese',
 'Chinese-Style',
 'Christmas',
 'Coco',
 'Coconut-Lentil',
 'Coconut-Lime',
 'Cocotte',
 'Colombian',
 'Cooker',
 'Cooks',
 'Cookup',
 'Costa',
 'Croatian',
 'Cuban',
 'Cuban-Inspired',
 'Cuban-Style',
 'Cubanos',
 'Curry-Coconut',
 'Czech',
 'Czechoslovakian',
 'Danielle',
 'Danish',
 'Dominican',
 'Dominican-Style',
 'Egyptian',
 'Elizabeth',
 'Ellen',
 'Elote',
 'Ethiopian',
 'Fasolakia',
 'Fasoulia',
 'Filipino',
 'Filipino-Style',
 'French',
 'French-Canadian',
 'French

In [1273]:
all_recipe_names[:10]

['Pan-Fried Asparagus',
 'Creamy Au Gratin Potatoes',
 'Super-Delicious Zuppa Toscana',
 'Simple Teriyaki Sauce',
 'Spicy Korean Fried Chicken with Gochujang Sauce',
 'Spaghetti Aglio e Olio',
 'Easy Garam Masala',
 'Easy Chorizo Street Tacos',
 'Russian Cabbage Rolls with Gravy',
 'Shrimp Scampi with Pasta']

In [1281]:
all_recipe_names2 = all_recipe_names.copy()
all_recipe_names2[:10]

['Pan-Fried Asparagus',
 'Creamy Au Gratin Potatoes',
 'Super-Delicious Zuppa Toscana',
 'Simple Teriyaki Sauce',
 'Spicy Korean Fried Chicken with Gochujang Sauce',
 'Spaghetti Aglio e Olio',
 'Easy Garam Masala',
 'Easy Chorizo Street Tacos',
 'Russian Cabbage Rolls with Gravy',
 'Shrimp Scampi with Pasta']

In [1282]:
# https://stackoverflow.com/questions/40291443/python-convert-a-string-to-lowercase-except-some-special-strings/40291577
lowerAllExcept = lambda x: " ".join( a if a in token_with_country_prefix else a.lower()
                                    for a in x.split() )

for i, recipe in enumerate(all_recipe_names2):
    for t in token_with_country_prefix:
        all_recipe_names2[i] = lowerAllExcept(all_recipe_names2[i])

In [1295]:
all_recipe_names_corpus = ("\n").join(all_recipe_names2)

recipe_tokens = list(set(nltk.word_tokenize(all_recipe_names_corpus)))

lower_recipe_tokens = []
for token in recipe_tokens:
    if token[0].islower():
        lower_recipe_tokens.append(token)
        
lower_recipe_tokens

['chunks',
 'pizzelle',
 'vinagrete',
 'podge',
 'torrone',
 'whit',
 'pasty',
 'farmer',
 'lebkuchen',
 'appetizers',
 'delight',
 'atsara',
 'lubia',
 'potato-lentil',
 'rellenos',
 'cordon',
 'horseback',
 'jigg',
 'lassi',
 'prosciutto',
 'bailey',
 'dulces',
 'original',
 'kroppkakor',
 'company',
 'machhali',
 'juleskringle',
 'tzatziki',
 'almond',
 'bayildi',
 'hasenpfeffer',
 'jay',
 'tonkatsu',
 'laotian',
 'pudim',
 'butter',
 'roasted',
 'breton',
 'maast-o',
 'veggie',
 'veneto',
 'jablkami',
 'gravy',
 'bangers',
 'champ',
 'yemista',
 'die',
 'shanghai',
 'long-grain',
 'shearers',
 'merah',
 'shiso',
 'kee',
 'bistecca',
 'piroshki',
 'oven-baked',
 'dijon',
 'danielle',
 'an',
 'giniling',
 'bramboracky',
 'ladolemono',
 'addictive',
 'sunken',
 'junka',
 'belle',
 'scaloppine',
 'chorizo',
 'goan',
 'chomeur',
 'yazdi',
 'kofta',
 'kick',
 'imam',
 'tuscan',
 'pastitsio',
 'mocha',
 'cobbler',
 'quebec',
 'before',
 'watermelon',
 'medianoche',
 'tso',
 'mom',
 'cotta

In [1296]:
len(lower_recipe_tokens)

3015

In [1297]:
upper_recipe_tokens = list(filter(str.istitle, recipe_tokens))
len(upper_recipe_tokens)

210

In [1298]:
upper_recipe_tokens

['Caribbean-Spiced',
 'Christmas',
 'Indian',
 'British',
 'Arabic',
 'Samosas',
 'Man',
 'Korean',
 'Filipino-Style',
 'Sarciado',
 'Malaysian',
 'Salvadorena',
 'Italiana',
 'Liberian',
 'Lankan',
 'Russian',
 'Cuban-Inspired',
 'Afghani',
 'Verdes',
 'Cambodian',
 'Southwest',
 'Mandazi',
 'Polynesian',
 'Romana',
 'Rican',
 'Fasoulia',
 'Jamaican',
 'Mongolian',
 'Zealand',
 'Bolivian',
 'Sandwiches',
 'Mandelmakronen',
 'Coconut-Lentil',
 'Singaporean',
 'Argentine',
 'Czechoslovakian',
 'Australian',
 'Sarma',
 'Helene',
 'Pierogi',
 'Indio',
 'Argentinean',
 'Sardine',
 'Saudi',
 'Newfoundland-Style',
 'Mandu',
 'Indonesian',
 "'Chinese",
 'Sans',
 'Mange',
 'Guatemalan',
 'Cubanos',
 'Island-Style',
 'Serbian',
 'Welsh',
 'Mexico',
 'Samoan',
 'Elote',
 'Chilaquiles',
 'Bermuda',
 'Japanese',
 'Martini',
 'Lanka',
 'Mozambique',
 'Bhutanese',
 'Rico',
 'Somali',
 'Roman',
 'Capezzoli',
 'Maltese',
 'Belgian',
 'Georgian',
 'Jan',
 'Guyanese',
 'Swedish',
 'Island',
 'Salvadoran

In [ ]:
# token_with_country_prefix = token_with_country_prefix.remove("No-Cook")

# for token in recipe_tokens:
#     if token[0].islower():
#         lower_recipe_tokens.append(token)
        
# len(lower_recipe_tokens)

In [ ]:
# token_with_country_prefix = []
# for rt in recipe_tokens:
#     for lan in lancester_c:
#         if lan in rt:
#             token_with_country_prefix.append(rt)

# token_with_country_prefix = sorted(list(set(token_with_country_prefix)))
# token_with_country_prefix

In [ ]:
# from nltk import edit_distance

# #Defining a function to get similar words with edit_distance < 3 from the corpus
# def is_similar_words(word, corpus, dist=3): 
#     similar_words = []
#     if word in corpus:
#         return
#     else:
#         w2 = word
#         similar_words = [w1 for w1 in corpus if edit_distance(w1, w2) < dist]

#     if len(similar_words) > 0:
#         return (word, similar_words)
    
#     return None

# is_similar_words("angola", countries)

In [ ]:
# dist_3_country_token = []
# for rt in recipe_tokens:
#     try:
#         result = is_similar_words(rt, lancester_c)
#         if result != None:
#             dist_3_country_token.append(result)
#     except Exception as e:
#         pass

# dist_3_country_token

In [ ]:
# p_ingredients[:10]

## Data merging and creating bigram

In [1299]:
all_recipe_names2[:10]

['pan-fried asparagus',
 'creamy au gratin potatoes',
 'super-delicious zuppa toscana',
 'simple teriyaki sauce',
 'spicy Korean fried chicken with gochujang sauce',
 'spaghetti aglio e olio',
 'easy garam masala',
 'easy chorizo street tacos',
 'Russian cabbage rolls with gravy',
 'shrimp scampi with pasta']

In [1300]:
p_ingredients[:10]

['4 tablespoons grated orange peel',
 '4 green bell pepper, cut into matchstick-size pieces',
 '4 cups dry bread crumbs',
 '4 teaspoon New Mexico chili powder',
 '4 teaspoon mayonnaise, or to taste',
 '4.4 tablespoons cornstarch',
 '4 ounces dry miniature ravioli',
 '4 cups peeled and cubed potatoes',
 '4 bunch fresh basil, divided',
 '4 sugar snap peas']

In [1309]:
def generate_bigram_from_entry(entry):
    bigrams = nltk.bigrams(entry.split(' '))
    frequence = nltk.FreqDist(bigrams)
    return dict(sorted(frequence.items(), key=lambda item: item[0]))

generate_bigram_from_entry("4 tablespoons grated orange peel")

{('4', 'tablespoons'): 1,
 ('grated', 'orange'): 1,
 ('orange', 'peel'): 1,
 ('tablespoons', 'grated'): 1}

In [1311]:
from collections import Counter

name_bigrams = {}

for name in all_recipe_names2:
    name_bigrams = dict(Counter(name_bigrams)+Counter(generate_bigram_from_entry(name)))
    
name_bigrams

{('pan-fried', 'asparagus'): 1,
 ('au', 'gratin'): 3,
 ('creamy', 'au'): 1,
 ('gratin', 'potatoes'): 1,
 ('super-delicious', 'zuppa'): 1,
 ('zuppa', 'toscana'): 3,
 ('simple', 'teriyaki'): 1,
 ('teriyaki', 'sauce'): 9,
 ('Korean', 'fried'): 5,
 ('chicken', 'with'): 24,
 ('fried', 'chicken'): 10,
 ('gochujang', 'sauce'): 3,
 ('spicy', 'Korean'): 4,
 ('with', 'gochujang'): 2,
 ('aglio', 'e'): 1,
 ('e', 'olio'): 1,
 ('spaghetti', 'aglio'): 1,
 ('easy', 'garam'): 1,
 ('garam', 'masala'): 3,
 ('chorizo', 'street'): 1,
 ('easy', 'chorizo'): 1,
 ('street', 'tacos'): 3,
 ('Russian', 'cabbage'): 5,
 ('cabbage', 'rolls'): 11,
 ('rolls', 'with'): 6,
 ('with', 'gravy'): 2,
 ('scampi', 'with'): 1,
 ('shrimp', 'scampi'): 7,
 ('with', 'pasta'): 2,
 ('and', 'potato'): 17,
 ('chicken', 'and'): 51,
 ('greek', 'lemon'): 6,
 ('lemon', 'chicken'): 4,
 ('potato', 'bake'): 3,
 ('easy', 'mexican'): 6,
 ('mexican', 'casserole'): 5,
 ('apple', 'cake'): 8,
 ('cake', 'i'): 3,
 ('german', 'apple'): 7,
 ('Spanish',

In [1312]:
ingre_bigrams = {}

for ingre in p_ingredients:
    ingre_bigrams = dict(Counter(ingre_bigrams)+Counter(generate_bigram_from_entry(ingre)))
    
ingre_bigrams

{('4', 'tablespoons'): 1348,
 ('grated', 'orange'): 9,
 ('orange', 'peel'): 8,
 ('tablespoons', 'grated'): 29,
 ('4', 'green'): 170,
 ('bell', 'pepper,'): 182,
 ('cut', 'into'): 990,
 ('green', 'bell'): 125,
 ('into', 'matchstick-size'): 6,
 ('matchstick-size', 'pieces'): 6,
 ('pepper,', 'cut'): 64,
 ('4', 'cups'): 1004,
 ('bread', 'crumbs'): 78,
 ('cups', 'dry'): 21,
 ('dry', 'bread'): 25,
 ('4', 'teaspoon'): 590,
 ('Mexico', 'chili'): 1,
 ('New', 'Mexico'): 9,
 ('chili', 'powder'): 41,
 ('teaspoon', 'New'): 2,
 ('mayonnaise,', 'or'): 6,
 ('or', 'to'): 534,
 ('teaspoon', 'mayonnaise,'): 1,
 ('to', 'taste'): 956,
 ('4.4', 'tablespoons'): 159,
 ('tablespoons', 'cornstarch'): 9,
 ('4', 'ounces'): 501,
 ('dry', 'miniature'): 1,
 ('miniature', 'ravioli'): 1,
 ('ounces', 'dry'): 15,
 ('and', 'cubed'): 88,
 ('cubed', 'potatoes'): 6,
 ('cups', 'peeled'): 12,
 ('peeled', 'and'): 494,
 ('4', 'bunch'): 105,
 ('basil,', 'divided'): 3,
 ('bunch', 'fresh'): 54,
 ('fresh', 'basil,'): 19,
 ('4', 'sug

In [1313]:
all_bigrams = {}

all_bigrams = dict(Counter(name_bigrams)+Counter(ingre_bigrams))

all_bigrams

{('pan-fried', 'asparagus'): 1,
 ('au', 'gratin'): 3,
 ('creamy', 'au'): 1,
 ('gratin', 'potatoes'): 1,
 ('super-delicious', 'zuppa'): 1,
 ('zuppa', 'toscana'): 3,
 ('simple', 'teriyaki'): 1,
 ('teriyaki', 'sauce'): 20,
 ('Korean', 'fried'): 5,
 ('chicken', 'with'): 25,
 ('fried', 'chicken'): 10,
 ('gochujang', 'sauce'): 3,
 ('spicy', 'Korean'): 4,
 ('with', 'gochujang'): 2,
 ('aglio', 'e'): 1,
 ('e', 'olio'): 1,
 ('spaghetti', 'aglio'): 1,
 ('easy', 'garam'): 1,
 ('garam', 'masala'): 17,
 ('chorizo', 'street'): 1,
 ('easy', 'chorizo'): 1,
 ('street', 'tacos'): 3,
 ('Russian', 'cabbage'): 5,
 ('cabbage', 'rolls'): 11,
 ('rolls', 'with'): 6,
 ('with', 'gravy'): 2,
 ('scampi', 'with'): 1,
 ('shrimp', 'scampi'): 7,
 ('with', 'pasta'): 2,
 ('and', 'potato'): 17,
 ('chicken', 'and'): 51,
 ('greek', 'lemon'): 6,
 ('lemon', 'chicken'): 4,
 ('potato', 'bake'): 3,
 ('easy', 'mexican'): 6,
 ('mexican', 'casserole'): 5,
 ('apple', 'cake'): 8,
 ('cake', 'i'): 3,
 ('german', 'apple'): 7,
 ('Spanish

In [1314]:
def find_dict_tuple_key(search):
    entry = {
        "token": search,
        "bigrams": []
    }
    bigrams = {x: all_bigrams[x] for x in all_bigrams.keys() if x[0] == search}
    for key, value in bigrams.items():
        newDict = {key[1]: value}
        entry["bigrams"].append(newDict)
    return entry

find_dict_tuple_key('spicy')

{'token': 'spicy',
 'bigrams': [{'Korean': 4},
  {'stir': 1},
  {'chicken': 4},
  {'szechuan': 1},
  {'basil': 2},
  {'vegan': 1},
  {'red': 2},
  {'Indian': 4},
  {'Vietnamese': 2},
  {'bok': 1},
  {'thai': 4},
  {'tuna': 2},
  {'stir-fry': 1},
  {'orange': 4},
  {'beef': 3},
  {'dipping': 1},
  {'pesto': 1},
  {'cabbage': 1},
  {'asian': 1},
  {'crispy': 1},
  {'eggplant': 4},
  {'green': 1},
  {'pork': 5},
  {'shrimp': 3},
  {'noodles': 1},
  {'Chinese': 2},
  {'Southwest': 1},
  {'asian-style': 1},
  {'sushi': 1},
  {'Mexican-American': 1},
  {'African': 1},
  {'yellowtail': 1},
  {'avocado': 1},
  {'Peruvian': 1},
  {'peach': 1},
  {'mango': 1},
  {'rice': 2},
  {'salmon': 1},
  {'yogurt': 1},
  {'Sinterklass': 1},
  {'Italian': 4},
  {'feta': 1},
  {'and': 1},
  {'banana': 1},
  {'penyet': 1},
  {'calabrian': 1},
  {'marinated': 1},
  {'tomato': 1},
  {'himalayan': 1},
  {'fried': 1},
  {'curry': 1},
  {'cilantro': 1},
  {'sesame': 1},
  {'brown': 2},
  {'Portuguese': 1},
  {'sea

In [1318]:
all_tokens = recipe_tokens + ingre_tokens

len(all_tokens)

6064

In [1319]:
bigram_in_list = []
for value in all_tokens:
    bigram_in_list.append(find_dict_tuple_key(value))
    
bigram_in_list

[{'token': 'chunks',
  'bigrams': [{'Dominican': 1},
   {'in': 1},
   {'(Optional)': 3},
   {'with': 2},
   {'fresh': 1}]},
 {'token': 'pizzelle', 'bigrams': []},
 {'token': 'vinagrete', 'bigrams': []},
 {'token': 'podge', 'bigrams': []},
 {'token': 'torrone', 'bigrams': []},
 {'token': 'whit', 'bigrams': []},
 {'token': 'pasty', 'bigrams': []},
 {'token': 'farmer', 'bigrams': []},
 {'token': 'Caribbean-Spiced', 'bigrams': [{'roast': 1}]},
 {'token': 'lebkuchen', 'bigrams': [{'men': 1}]},
 {'token': 'appetizers', 'bigrams': [{'for': 1}]},
 {'token': 'delight', 'bigrams': []},
 {'token': 'atsara', 'bigrams': []},
 {'token': 'Christmas',
  'bigrams': [{'cookies': 3},
   {'brunch': 1},
   {'glogg': 1},
   {'plum': 1},
   {'red': 1},
   {'cabbage': 1},
   {'coconut': 1},
   {'stollen': 1},
   {'lefse': 1},
   {'dinner': 1},
   {'baklava': 1},
   {'cake': 1},
   {'eve': 1},
   {'cakes': 2},
   {'kringle': 1}]},
 {'token': 'lubia', 'bigrams': [{'polo': 1}]},
 {'token': 'potato-lentil', 'bigr

## Add Phonetics

In [1320]:
!pipenv install eng-to-ipa 

Installing eng-to-ipa...

Installing dependencies from Pipfile.lock (577ce1)...
Ignoring argcomplete: markers 'python_full_version < "3.8.0"' don't match your environment
Ignoring importlib-metadata: markers 'python_version == "3.7" and python_full_version < "3.8.0" and python_full_version < "3.8.0" and python_full_version < "3.8.0"' don't match your environment
Ignoring typing-extensions: markers 'python_full_version < "3.8.0"' don't match your environment


Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.


[    ] Installing..
[=   ] Installing eng-to-ipa..
[==  ] Installing eng-to-ipa..
[=== ] Installing eng-to-ipa..
[ ===] Installing eng-to-ipa..
[  ==] Installing eng-to-ipa..
[   =] Installing eng-to-ipa..
[    ] Installing eng-to-ipa..
[   =] Installing eng-to-ipa..
[  ==] Installing eng-to-ipa..
[ ===] Installing eng-to-ipa..
[====] Installing eng-to-ipa..
[=== ] Installing eng-to-ipa..
[==  ] Installing eng-to-ipa..
[=   ] Installing eng-to-ipa..
[    ] Installing eng-to-ipa..
[=   ] Installing eng-to-ipa..
[==  ] Installing eng-to-ipa..
[=== ] Installing eng-to-ipa..
[ ===] Installing eng-to-ipa..
[  ==] Installing eng-to-ipa..
[   =] Installing eng-

In [1323]:
import eng_to_ipa as eng_to_ipa

eng_to_ipa.convert("hey!")

'heɪ!'

In [1324]:
for bigram in bigram_in_list:
    try:
        bigram["ipa"] = eng_to_ipa.convert(eng_to_ipa.convert(bigram["token"]))
    except Exception as e:
        pass

bigram_in_list

[{'token': 'chunks',
  'bigrams': [{'Dominican': 1},
   {'in': 1},
   {'(Optional)': 3},
   {'with': 2},
   {'fresh': 1}],
  'ipa': 'ʧəŋʧəŋks*'},
 {'token': 'pizzelle', 'bigrams': [], 'ipa': 'pizzelle**'},
 {'token': 'vinagrete', 'bigrams': [], 'ipa': 'vinagrete**'},
 {'token': 'podge', 'bigrams': [], 'ipa': 'pɑʤ*ɑʤ'},
 {'token': 'torrone', 'bigrams': [], 'ipa': 'torrone**'},
 {'token': 'whit', 'bigrams': [], 'ipa': 'wɪt*'},
 {'token': 'pasty', 'bigrams': [], 'ipa': 'pasty**'},
 {'token': 'farmer', 'bigrams': [], 'ipa': 'ˈˈfɑrmər*'},
 {'token': 'Caribbean-Spiced',
  'bigrams': [{'roast': 1}],
  'ipa': 'caribbean-spiced**'},
 {'token': 'lebkuchen', 'bigrams': [{'men': 1}], 'ipa': 'lebkuchen**'},
 {'token': 'appetizers', 'bigrams': [{'for': 1}], 'ipa': 'ˈæˈæpəˌtaɪzərz*'},
 {'token': 'delight', 'bigrams': [], 'ipa': 'dɪˈlaɪt*'},
 {'token': 'atsara', 'bigrams': [], 'ipa': 'atsara**'},
 {'token': 'Christmas',
  'bigrams': [{'cookies': 3},
   {'brunch': 1},
   {'glogg': 1},
   {'plum': 1},
 

## Old code (need to be changed / replaced later on)

Flatten each data entry into a string

In [ ]:
corpus_list = []
for item in recipes:
    item['ingredients']=','.join(item['ingredients'])
    try:
        item['text'] = item['name'] + " " + item["ingredients"]
    except Exception as e:
        item['name'] = ""
        item['text'] = item['name'] + " " + item["ingredients"]
        print(e)
    corpus_list.append(item['text'])
    
corpus_list[:3]

Convert entire flattened list into a string

In [ ]:
corpus = ','.join(corpus_list)

Compute bigram

In [ ]:
import nltk
nltk.download('punkt')

tokens = nltk.word_tokenize(corpus)
bigrams = nltk.bigrams(tokens)
frequence = nltk.FreqDist(bigrams)
for key,value in frequence.items():
    print(key,value)

In [ ]:
len(tokens)

Convert bigrams into dictionaries, with bigram as key, frequency as value

In [ ]:
result = dict(sorted(frequence.items(), key=lambda item: item[0]))
result

Get unique tokens and sort them in an ascending order

In [ ]:
unique_tokens = sorted(list(set(tokens)))
unique_tokens

Combine bigrams of the same first word into a dictionary

In [ ]:
def find_dict_tuple_key(search):
    entry = {
        "token": search,
        "bigrams": []
    }
    bigrams = {x: result[x] for x in result.keys() if x[0] == search}
    for key, value in bigrams.items():
        newDict = {key[1]: value}
        entry["bigrams"].append(newDict)
    return entry

find_dict_tuple_key('Garlic')

Do the same to all the tokens to create a list of dictionaries

In [ ]:
bigram_list = []
for value in unique_tokens:
    bigram_list.append(find_dict_tuple_key(value))
    
bigram_list

In [ ]:
len(bigram_list)

In [ ]:
len(unique_tokens)

## Numbers and placeholder

## POS tagging

# Create edit distance

# Create bigram

#Chunking/Phrases
